<a href="https://colab.research.google.com/github/Benxperia/Dissertation/blob/Climate/climate_data_v9.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Integrated Climate Data Retrieval Script

This script combines data from multiple climate data sources (NOAA, ERA5, MET Norway/Frost,
SeNorge, NVE, CHELSA, and E-OBS) to create the most complete dataset possible for
Norwegian glacier study sites.


In [ ]:
!pip install netCDF4

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.3/9.3 MB 49.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 63.8 MB/s eta 0:00:00


In [ ]:
# Import libraries
import os
import sys
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
import requests
from scipy import stats
import xarray as xr
import netCDF4 as nc
import zipfile
import time
import warnings
import io
from shapely.geometry import Point
import json
from urllib.parse import urlencode
from google.colab import drive
from google.colab import files

In [ ]:
# Mount Google Drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
# Suppress warnings
warnings.filterwarnings('ignore')

In [ ]:
def upload_glacier_sites(file_path):
    try:
        sites_df = pd.read_csv(file_path)
        # Rename the first column to 'site_name'
        sites_df = sites_df.rename(columns={sites_df.columns[0]: 'site_name'})
        # Access latitude and longitude by column index (assuming they are in columns 1 and 2)
        sites_df = sites_df.rename(columns={sites_df.columns[1]: 'latitude', sites_df.columns[2]: 'longitude'})
        return sites_df
    except Exception as e:
        print(f"Error loading glacier sites: {str(e)}")
        return None

def setup_cds_api():
    """
    Setup Climate Data Store (CDS) API credentials.

    Returns:
    -------
    bool
        True if setup was successful, False otherwise
    """
    try:
        # Install the CDS API client if needed
        import os

        # Check for API key file
        cds_key_file = os.path.expanduser('~/.cdsapirc')
        if os.path.exists(cds_key_file):
            print("CDS API configuration file already exists.")
            # Force recreation
            recreate = input("Would you like to recreate it? (y/n): ").lower() == 'y'
            if not recreate:
                return True

        print("\nCDS API configuration")
        print("Please register at https://cds.climate.copernicus.eu/user/register")
        print("Then go to https://cds.climate.copernicus.eu/api-how-to and copy your API key")

        url = input("Enter CDS API URL (default: https://cds.climate.copernicus.eu/api/v2): ")
        if not url:
            url = "https://cds.climate.copernicus.eu/api/v2"

        key = input("Enter your CDS API key: ")

        with open(cds_key_file, 'w') as f:
            f.write(f"url: {url}\n")
            f.write(f"key: {key}\n")

        os.chmod(cds_key_file, 0o600)
        print(f"CDS API configuration saved to {cds_key_file}")
        return True
    except Exception as e:
        print(f"Error setting up CDS API: {str(e)}")
        return False

In [ ]:
def verify_data_availability(sites_df, noaa_token=None, cds_setup=False, frost_client_id=None):
    """
    Verify which variables are available from each data source for each site.

    Parameters:
    ----------
    sites_df : pandas.DataFrame
        DataFrame with glacier site information
    noaa_token : str, optional
        NOAA API token
    cds_setup : bool, optional
        Whether CDS API is set up
    frost_client_id : str, optional
        MET Norway Frost API client ID

    Returns:
    -------
    dict
        Dictionary of availability by site and source
    """
    availability = {}

    for i, site in sites_df.iterrows():
        site_name = site['site_name']
        lat = site['latitude']
        lon = site['longitude']

        print(f"Checking data availability for site: {site_name} ({lat}, {lon})")

        availability[site_name] = {
            'noaa': {'available': False, 'variables': {}},
            'era5': {'available': False, 'variables': {}},
            'frost': {'available': False, 'variables': {}},
            'senorge': {'available': False, 'variables': {}},
            'nve': {'available': False, 'variables': {}},
            'chelsa': {'available': False, 'variables': {}},
            'eobs': {'available': False, 'variables': {}}
        }

        # Check NOAA availability if token provided
        if noaa_token:
            headers = {'token': noaa_token}
            base_url = "https://www.ncdc.noaa.gov/cdo-web/api/v2"

            # Find nearby stations
            station_url = f"{base_url}/stations"
            params = {
                'extent': f"{lat-1},{lon-1},{lat+1},{lon+1}",
                'limit': 1000
            }

            try:
                print(f"  Checking NOAA stations near {site_name}...")
                response = requests.get(station_url, headers=headers, params=params, timeout=30)
                if response.status_code == 200:
                    stations = response.json()
                    if 'results' in stations and len(stations['results']) > 0:
                        # Found stations, now check data types
                        closest_station = min(
                            stations['results'],
                            key=lambda s: ((lat - float(s['latitude']))**2 + (lon - float(s['longitude']))**2)**0.5
                        )
                        station_id = closest_station['id']
                        station_name = closest_station['name']
                        station_lat = closest_station['latitude']
                        station_lon = closest_station['longitude']

                        print(f"  Found station: {station_name} ({station_lat}, {station_lon})")

                        # Check for each data type with a small date range
                        variables = ['PRCP', 'TMAX', 'TMIN', 'AWND', 'SNOW', 'SNWD', 'ACSH']

                        for var in variables:
                            data_url = f"{base_url}/data"
                            # Just check most recent complete year
                            current_year = datetime.now().year - 1
                            data_params = {
                                'datasetid': 'GHCND',
                                'stationid': station_id,
                                'startdate': f"{current_year}-01-01",
                                'enddate': f"{current_year}-12-31",
                                'datatypeid': var,
                                'limit': 10
                            }

                            try:
                                var_response = requests.get(data_url, headers=headers, params=data_params, timeout=30)
                                if var_response.status_code == 200:
                                    var_data = var_response.json()
                                    if 'results' in var_data and var_data['results']:
                                        availability[site_name]['noaa']['variables'][var] = True
                                        print(f"    Variable {var}: Available")
                                    else:
                                        availability[site_name]['noaa']['variables'][var] = False
                                        print(f"    Variable {var}: Not available")
                                else:
                                    availability[site_name]['noaa']['variables'][var] = False
                                    print(f"    Variable {var}: Not available (API error)")
                            except Exception as e:
                                availability[site_name]['noaa']['variables'][var] = False
                                print(f"    Variable {var}: Error - {str(e)}")

                        availability[site_name]['noaa']['available'] = True
                        availability[site_name]['noaa']['station_id'] = station_id
                        availability[site_name]['noaa']['station_name'] = station_name
                        availability[site_name]['noaa']['station_lat'] = station_lat
                        availability[site_name]['noaa']['station_lon'] = station_lon
                    else:
                        print(f"  No NOAA stations found near {site_name}")
                else:
                    print(f"  NOAA API error: {response.status_code} - {response.text}")
            except Exception as e:
                print(f"  Error checking NOAA data: {str(e)}")

        # Check ERA5 availability if CDS is set up
        if cds_setup:
            try:
                print(f"  Checking ERA5 data availability for {site_name}...")
                # ERA5 typically has global coverage, so we just need to verify API access
                import cdsapi
                try:
                    c = cdsapi.Client()
                    # Just check if the client can be initialized
                    availability[site_name]['era5']['available'] = True
                    availability[site_name]['era5']['variables'] = {
                        't2m': True,  # Temperature
                        'tp': True    # Precipitation
                    }
                    print(f"  ERA5 data available for {site_name}")
                    print(f"    Variable t2m (temperature): Available")
                    print(f"    Variable tp (precipitation): Available")
                except Exception as e:
                    print(f"  Error initializing CDS client: {str(e)}")
                    print(f"  ERA5 data not available for {site_name} due to client initialization error")
            except Exception as e:
                print(f"  Error checking ERA5 data: {str(e)}")

        # Check MET Norway Frost API availability
        if frost_client_id:
            try:
                print(f"  Checking MET Norway Frost data availability for {site_name}...")

                # Find nearby weather stations
                stations_url = "https://frost.met.no/sources/v0.jsonld"
                params = {
                    'geometry': f'nearest(POINT({lon} {lat}))',
                    'nearestmaxcount': 5,
                    'types': 'SN',  # Standard Norwegian weather stations
                }

                r = requests.get(stations_url, params=params, auth=(frost_client_id, ''))

                if r.status_code == 200:
                    stations_data = r.json()

                    if 'data' in stations_data and stations_data['data']:
                        station = stations_data['data'][0]
                        station_id = station['id']
                        station_name = station['name']

                        # Check which elements are available
                        elements_url = f"https://frost.met.no/observations/availableTimeSeries/v0.jsonld"
                        elements_params = {
                            'sources': station_id
                        }

                        r = requests.get(elements_url, params=elements_params, auth=(frost_client_id, ''))

                        if r.status_code == 200:
                            elements_data = r.json()

                            # Map element codes to more readable names
                            element_mapping = {
                                'air_temperature': 'temperature',
                                'mean(air_temperature P1D)': 'mean_temperature',
                                'min(air_temperature P1D)': 'min_temperature',
                                'max(air_temperature P1D)': 'max_temperature',
                                'sum(precipitation_amount P1D)': 'precipitation',
                                'snow_depth': 'snow_depth',
                                'wind_speed': 'wind_speed',
                                'max(wind_speed P1D)': 'max_wind_speed'
                            }

                            # Check each relevant element
                            if 'data' in elements_data:
                                found_elements = set()
                                for item in elements_data['data']:
                                    element_id = item.get('elementId')
                                    if element_id in element_mapping:
                                        found_elements.add(element_id)

                                for element_id, variable_name in element_mapping.items():
                                    if element_id in found_elements:
                                        availability[site_name]['frost']['variables'][variable_name] = True
                                        print(f"    Variable {variable_name}: Available")
                                    else:
                                        availability[site_name]['frost']['variables'][variable_name] = False
                                        print(f"    Variable {variable_name}: Not available")

                            availability[site_name]['frost']['available'] = True
                            availability[site_name]['frost']['station_id'] = station_id
                            availability[site_name]['frost']['station_name'] = station_name
                    else:
                        print(f"  No Frost stations found near {site_name}")
                else:
                    print(f"  Frost API error: {r.status_code} - {r.text}")
            except Exception as e:
                print(f"  Error checking Frost data: {str(e)}")

        # Check SeNorge data availability - This is a gridded product covering all of Norway
        # so it should be available for all Norwegian sites
        try:
            print(f"  Checking SeNorge data availability for {site_name}...")

            # SeNorge data is available via Thredds
            # For simplicity, we'll just check if the site is within Norway's bounding box
            norway_bounds = {
                'min_lat': 57.5,
                'max_lat': 71.5,
                'min_lon': 4.0,
                'max_lon': 31.5
            }

            if (norway_bounds['min_lat'] <= lat <= norway_bounds['max_lat'] and
                norway_bounds['min_lon'] <= lon <= norway_bounds['max_lon']):

                availability[site_name]['senorge']['available'] = True
                availability[site_name]['senorge']['variables'] = {
                    'temperature': True,
                    'precipitation': True,
                    'snow_depth': True
                }

                print(f"  SeNorge data available for {site_name}")
                print(f"    Variable temperature: Available")
                print(f"    Variable precipitation: Available")
                print(f"    Variable snow_depth: Available")
            else:
                print(f"  SeNorge data not available for {site_name} (outside Norway)")
        except Exception as e:
            print(f"  Error checking SeNorge data: {str(e)}")

        # Check NVE glacier data availability - This requires knowing the glacier name
        try:
            print(f"  Checking NVE glacier data availability for {site_name}...")

            # For simplicity, we'll just check if we can access the NVE API
            # In a real scenario, you would search for the specific glacier by name
            nve_api_url = "https://api.nve.no/hydrology/glacier"

            try:
                # Try to access the API
                r = requests.get(nve_api_url, timeout=10)

                if r.status_code == 200:
                    availability[site_name]['nve']['available'] = True
                    availability[site_name]['nve']['variables'] = {
                        'mass_balance': True,
                        'length_change': True,
                        'terminus_position': True
                    }

                    print(f"  NVE glacier data API is accessible")
                    print(f"    Variable mass_balance: Available (if glacier is monitored)")
                    print(f"    Variable length_change: Available (if glacier is monitored)")
                    print(f"    Variable terminus_position: Available (if glacier is monitored)")
                else:
                    print(f"  NVE API error: {r.status_code}")
            except Exception as e:
                print(f"  Cannot access NVE API: {str(e)}")
        except Exception as e:
            print(f"  Error checking NVE data: {str(e)}")

        # Check CHELSA data availability - Global coverage
        try:
            print(f"  Checking CHELSA data availability for {site_name}...")

            # CHELSA is a global dataset, so it should be available for all sites
            # We'll just check if we can access the API
            chelsa_url = "https://chelsa-climate.org/downloads/"

            try:
                # Try to access the website
                r = requests.get(chelsa_url, timeout=10)

                if r.status_code == 200:
                    availability[site_name]['chelsa']['available'] = True
                    availability[site_name]['chelsa']['variables'] = {
                        'temperature': True,
                        'precipitation': True
                    }

                    print(f"  CHELSA data available for {site_name}")
                    print(f"    Variable temperature: Available")
                    print(f"    Variable precipitation: Available")
                else:
                    print(f"  CHELSA website error: {r.status_code}")
            except Exception as e:
                print(f"  Cannot access CHELSA website: {str(e)}")
        except Exception as e:
            print(f"  Error checking CHELSA data: {str(e)}")

        # Check E-OBS data availability - European coverage
        try:
            print(f"  Checking E-OBS data availability for {site_name}...")

            # E-OBS covers Europe, so check if site is within Europe's bounding box
            europe_bounds = {
                'min_lat': 25.0,
                'max_lat': 75.0,
                'min_lon': -25.0,
                'max_lon': 45.0
            }

            if (europe_bounds['min_lat'] <= lat <= europe_bounds['max_lat'] and
                europe_bounds['min_lon'] <= lon <= europe_bounds['max_lon']):

                availability[site_name]['eobs']['available'] = True
                availability[site_name]['eobs']['variables'] = {
                    'temperature': True,
                    'precipitation': True
                }

                print(f"  E-OBS data available for {site_name}")
                print(f"    Variable temperature: Available")
                print(f"    Variable precipitation: Available")
            else:
                print(f"  E-OBS data not available for {site_name} (outside Europe)")
        except Exception as e:
            print(f"  Error checking E-OBS data: {str(e)}")

    return availability

In [ ]:
def get_frost_data(sites_df, availability, client_id, start_year=1970, end_year=None, output_dir="/content/climate_outputs"):
    """
    Fetch data from the Norwegian Meteorological Institute's Frost API.

    Parameters:
    ----------
    sites_df : pandas.DataFrame
        DataFrame with glacier site information
    availability : dict
        Dictionary of availability by site and source
    client_id : str
        Frost API client ID
    start_year : int, optional
        Start year for data retrieval
    end_year : int, optional
        End year for data retrieval
    output_dir : str, optional
        Directory to save output files

    Returns:
    -------
    dict
        Dictionary with Frost climate data for each site
    """
    if end_year is None:
        end_year = datetime.now().year

    frost_results = {}

    # Create output directory for raw data
    frost_dir = f"{output_dir}/frost"
    os.makedirs(frost_dir, exist_ok=True)

    for i, site in sites_df.iterrows():
        site_name = site['site_name']
        lat = site['latitude']
        lon = site['longitude']

        if not availability[site_name]['frost']['available']:
            print(f"Frost data not available for {site_name}, skipping...")
            continue

        print(f"Fetching Frost data for site: {site_name} ({lat}, {lon})")

        station_id = availability[site_name]['frost']['station_id']
        station_name = availability[site_name]['frost']['station_name']

        frost_results[site_name] = {
            'station_info': {
                'id': station_id,
                'name': station_name
            },
            'data': {}
        }

        # Define the elements to retrieve based on availability
        available_elements = [elem for elem, is_available in
                            availability[site_name]['frost']['variables'].items()
                            if is_available]

        # Map variable names to Frost element IDs
        element_mapping = {
            'temperature': 'air_temperature',
            'mean_temperature': 'mean(air_temperature P1D)',
            'min_temperature': 'min(air_temperature P1D)',
            'max_temperature': 'max(air_temperature P1D)',
            'precipitation': 'sum(precipitation_amount P1D)',
            'snow_depth': 'snow_depth',
            'wind_speed': 'wind_speed',
            'max_wind_speed': 'max(wind_speed P1D)'
        }

        # Get the corresponding Frost element IDs
        elements = [element_mapping[elem] for elem in available_elements if elem in element_mapping]

        if not elements:
            print(f"  No available elements for {site_name}, skipping...")
            continue

        # Join elements with comma
        elements_str = ','.join(elements)

        # Fetch data for each year (to avoid timeout issues)
        for year in range(start_year, end_year + 1):
            try:
                # Define the time range for this year
                from_date = f"{year}-01-01"
                to_date = f"{year}-12-31"

                print(f"  Fetching data for year {year}...")

                # Set up the API request
                url = "https://frost.met.no/observations/v0.jsonld"
                params = {
                    'sources': station_id,
                    'elements': elements_str,
                    'referencetime': f"{from_date}/{to_date}",
                    'timeresolution': 'P1D',  # Daily data
                    'fields': 'value,elementId,unit,referenceTime'
                }

                # Make the request
                r = requests.get(url, params=params, auth=(client_id, ''))

                if r.status_code == 200:
                    data = r.json()

                    if 'data' in data:
                        # Store the data
                        frost_results[site_name]['data'][year] = data['data']
                        print(f"    Retrieved {len(data['data'])} records")
                    else:
                        print(f"    No data returned for {year}")
                else:
                    print(f"    API error for {year}: {r.status_code} - {r.text}")
            except Exception as e:
                print(f"    Error fetching data for {year}: {str(e)}")

            # Add a small delay to avoid rate limiting
            time.sleep(0.5)

        # Convert to DataFrame for easier processing
        all_data = []
        for year, year_data in frost_results[site_name]['data'].items():
            for obs in year_data:
                for obs_data in obs.get('observations', []):
                    record = {
                        'date': obs.get('referenceTime'),
                        'element': obs_data.get('elementId'),
                        'value': obs_data.get('value'),
                        'unit': obs_data.get('unit')
                    }
                    all_data.append(record)

        df = pd.DataFrame(all_data)

        # Save raw data
        if not df.empty:
            raw_file = f"{frost_dir}/{site_name}_frost_raw.csv"
            df.to_csv(raw_file, index=False)
            print(f"  Raw Frost data saved to: {raw_file}")

            # Store the DataFrame
            frost_results[site_name]['dataframe'] = df

    return frost_results

In [ ]:
def get_senorge_data(sites_df, availability, start_year=1970, end_year=None, output_dir="/content/climate_outputs"):
    if end_year is None:
        end_year = datetime.now().year

    senorge_results = {}

    # Create output directory for raw data
    senorge_dir = f"{output_dir}/senorge"
    os.makedirs(senorge_dir, exist_ok=True)

    for i, site in sites_df.iterrows():
        site_name = site['site_name']
        lat = site['latitude']
        lon = site['longitude']

        if not availability[site_name]['senorge']['available']:
            print(f"SeNorge data not available for {site_name}, skipping...")
            continue

        print(f"Fetching SeNorge data for site: {site_name} ({lat}, {lon})")

        senorge_results[site_name] = {
            'temperature': {},
            'precipitation': {},
            'snow_depth': {}
        }

        # SeNorge data is accessible through Thredds
        # For each variable, we need to access a different dataset

        # Before trying to access the Thredds server
        print(f"\n==== DEBUGGING: Accessing SeNorge data for {site_name} ====")
        try:
            # Create a test request to the Thredds server
            thredds_test_url = "https://thredds.met.no/thredds/catalog.html"
            print(f"  Testing connection to Thredds server: {thredds_test_url}")
            test_response = requests.get(thredds_test_url, timeout=10)
            print(f"  Connection test status code: {test_response.status_code}")

            # When constructing the data URL
            base_url = "https://thredds.met.no/thredds/dodsC/senorge/seNorge_2018/Archive"
            print(f"  Attempting to access SeNorge base URL: {base_url}")

            # After retrieving data (or simulating it)
            print(f"  SeNorge data processing complete")
        except Exception as e:
            print(f"  Exception during SeNorge data access: {str(e)}")
            print(f"  Exception type: {type(e).__name__}")

        # Temperature data
        if availability[site_name]['senorge']['variables'].get('temperature', False):
            print(f"  Fetching temperature data...")

            try:
                # Using newer seNorge2018 dataset
                base_url = "https://thredds.met.no/thredds/dodsC/senorge/seNorge_2018/Archive/daily-mean-temperature"

                # Create a list to store annual data
                temp_data = []

                for year in range(start_year, end_year + 1):
                    try:
                        print(f"    Processing year {year}...")
                        url = f"{base_url}/{year}/seNorge2018_temperature_{year}.nc"

                        # Open the dataset
                        ds = xr.open_dataset(url)

                        # Extract data for the site location
                        # seNorge uses a different coordinate system, so we need to convert
                        # For simplicity, we'll use the nearest grid point
                        site_data = ds.sel(longitude=lon, latitude=lat, method='nearest')

                        # Convert to DataFrame
                        df = site_data['tm'].to_dataframe().reset_index()

                        # Add to our list
                        temp_data.append(df)

                        # Close the dataset
                        ds.close()
                    except Exception as e:
                        print(f"    Error processing temperature data for {year}: {str(e)}")

                if temp_data:
                    # Combine all years
                    temp_df = pd.concat(temp_data)

                    # Save to file
                    temp_file = f"{senorge_dir}/{site_name}_senorge_temperature.csv"
                    temp_df.to_csv(temp_file, index=False)
                    print(f"  Temperature data saved to: {temp_file}")

                    # Store in results
                    senorge_results[site_name]['temperature'] = temp_df
            except Exception as e:
                print(f"  Error fetching temperature data: {str(e)}")

        # Precipitation data
        if availability[site_name]['senorge']['variables'].get('precipitation', False):
            print(f"  Fetching precipitation data...")

            try:
                # Using newer seNorge2018 dataset
                base_url = "https://thredds.met.no/thredds/dodsC/senorge/seNorge_2018/Archive/daily-total-precipitation"

                # Create a list to store annual data
                precip_data = []

                for year in range(start_year, end_year + 1):
                    try:
                        print(f"    Processing year {year}...")
                        url = f"{base_url}/{year}/seNorge2018_precipitation_{year}.nc"

                        # Open the dataset
                        ds = xr.open_dataset(url)

                        # Extract data for the site location
                        site_data = ds.sel(longitude=lon, latitude=lat, method='nearest')

                        # Convert to DataFrame
                        df = site_data['rr'].to_dataframe().reset_index()

                        # Add to our list
                        precip_data.append(df)

                        # Close the dataset
                        ds.close()
                    except Exception as e:
                        print(f"    Error processing precipitation data for {year}: {str(e)}")

                if precip_data:
                    # Combine all years
                    precip_df = pd.concat(precip_data)

                    # Save to file
                    precip_file = f"{senorge_dir}/{site_name}_senorge_precipitation.csv"
                    precip_df.to_csv(precip_file, index=False)
                    print(f"  Precipitation data saved to: {precip_file}")

                    # Store in results
                    senorge_results[site_name]['precipitation'] = precip_df
            except Exception as e:
                print(f"  Error fetching precipitation data: {str(e)}")

        # Snow depth data
        if availability[site_name]['senorge']['variables'].get('snow_depth', False):
            print(f"  Fetching snow depth data...")

            try:
                # Using newer seNorge2018 dataset
                base_url = "https://thredds.met.no/thredds/dodsC/senorge/seNorge_2018/Archive/daily-snow-depth"

                # Create a list to store annual data
                snow_data = []

                for year in range(start_year, end_year + 1):
                    try:
                        print(f"    Processing year {year}...")
                        url = f"{base_url}/{year}/seNorge2018_snowdepth_{year}.nc"

                        # Open the dataset
                        ds = xr.open_dataset(url)

                        # Extract data for the site location
                        site_data = ds.sel(longitude=lon, latitude=lat, method='nearest')

                        # Convert to DataFrame
                        df = site_data['sd'].to_dataframe().reset_index()

                        # Add to our list
                        snow_data.append(df)

                        # Close the dataset
                        ds.close()
                    except Exception as e:
                        print(f"    Error processing snow depth data for {year}: {str(e)}")

                if snow_data:
                    # Combine all years
                    snow_df = pd.concat(snow_data)

                    # Save to file
                    snow_file = f"{senorge_dir}/{site_name}_senorge_snow_depth.csv"
                    snow_df.to_csv(snow_file, index=False)
                    print(f"  Snow depth data saved to: {snow_file}")

                    # Store in results
                    senorge_results[site_name]['snow_depth'] = snow_df
            except Exception as e:
                print(f"  Error fetching snow depth data: {str(e)}")

    return senorge_results

In [ ]:
def get_nve_glacier_data(sites_df, availability, start_year=1970, end_year=None, output_dir="/content/climate_outputs"):
    """
    Fetch glacier-specific data from NVE's glacier database.

    Parameters:
    ----------
    sites_df : pandas.DataFrame
        DataFrame with glacier site information
    availability : dict
        Dictionary of availability by site and source
    start_year : int, optional
        Start year for data retrieval
    end_year : int, optional
        End year for data retrieval
    output_dir : str, optional
        Directory to save output files

    Returns:
    -------
    dict
        Dictionary with NVE glacier data for each site
    """
    if end_year is None:
        end_year = datetime.now().year

    nve_results = {}

    # Create output directory for raw data
    nve_dir = f"{output_dir}/nve"
    os.makedirs(nve_dir, exist_ok=True)

    for i, site in sites_df.iterrows():
        site_name = site['site_name']

        if not availability[site_name]['nve']['available']:
            print(f"NVE glacier data not available for {site_name}, skipping...")
            continue

        print(f"Fetching NVE glacier data for site: {site_name}")

        nve_results[site_name] = {
            'mass_balance': None,
            'length_change': None,
            'terminus_position': None
        }

        try:
            # Search for glacier by name
            # Note: The actual NVE API might have a different structure
            search_url = f"https://api.nve.no/hydrology/glacier/search?name={site_name}"

            r = requests.get(search_url)

            if r.status_code == 200:
                search_data = r.json()

                if search_data and len(search_data) > 0:
                    # Found a matching glacier
                    glacier_id = search_data[0]['id']
                    glacier_name = search_data[0]['name']

                    print(f"  Found glacier: {glacier_name} (ID: {glacier_id})")

                    # Get mass balance data
                    if availability[site_name]['nve']['variables'].get('mass_balance', False):
                        balance_url = f"https://api.nve.no/hydrology/glacier/{glacier_id}/massbalance?from={start_year}&to={end_year}"

                        r = requests.get(balance_url)

                        if r.status_code == 200:
                            balance_data = r.json()

                            # Convert to DataFrame
                            if balance_data:
                                balance_df = pd.DataFrame(balance_data)

                                # Save to file
                                balance_file = f"{nve_dir}/{site_name}_nve_mass_balance.csv"
                                balance_df.to_csv(balance_file, index=False)
                                print(f"  Mass balance data saved to: {balance_file}")

                                # Store in results
                                nve_results[site_name]['mass_balance'] = balance_df

                    # Get length change data
                    if availability[site_name]['nve']['variables'].get('length_change', False):
                        length_url = f"https://api.nve.no/hydrology/glacier/{glacier_id}/length?from={start_year}&to={end_year}"

                        r = requests.get(length_url)

                        if r.status_code == 200:
                            length_data = r.json()

                            # Convert to DataFrame
                            if length_data:
                                length_df = pd.DataFrame(length_data)

                                # Save to file
                                length_file = f"{nve_dir}/{site_name}_nve_length_change.csv"
                                length_df.to_csv(length_file, index=False)
                                print(f"  Length change data saved to: {length_file}")

                                # Store in results
                                nve_results[site_name]['length_change'] = length_df

                    # Get terminus position data
                    if availability[site_name]['nve']['variables'].get('terminus_position', False):
                        terminus_url = f"https://api.nve.no/hydrology/glacier/{glacier_id}/terminus?from={start_year}&to={end_year}"

                        r = requests.get(terminus_url)

                        if r.status_code == 200:
                            terminus_data = r.json()

                            # Convert to DataFrame
                            if terminus_data:
                                terminus_df = pd.DataFrame(terminus_data)

                                # Save to file
                                terminus_file = f"{nve_dir}/{site_name}_nve_terminus_position.csv"
                                terminus_df.to_csv(terminus_file, index=False)
                                print(f"  Terminus position data saved to: {terminus_file}")

                                # Store in results
                                nve_results[site_name]['terminus_position'] = terminus_df
                else:
                    print(f"  No matching glacier found for {site_name}")
            else:
                print(f"  API error: {r.status_code} - {r.text}")
        except Exception as e:
            print(f"  Error fetching NVE data: {str(e)}")

    return nve_results

In [ ]:
def get_chelsa_data(sites_df, availability, output_dir="/content/climate_outputs"):
    """
    Fetch climate data from CHELSA dataset.

    Parameters:
    ----------
    sites_df : pandas.DataFrame
        DataFrame with glacier site information
    availability : dict
        Dictionary of availability by site and source
    output_dir : str, optional
        Directory to save output files

    Returns:
    -------
    dict
        Dictionary with CHELSA climate data for each site
    """
    chelsa_results = {}

    # Create output directory for raw data
    chelsa_dir = f"{output_dir}/chelsa"
    os.makedirs(chelsa_dir, exist_ok=True)

    # CHELSA V2.1 uses 1981-2010 as the reference period
    reference_period = "1981-2010"

    for i, site in sites_df.iterrows():
        site_name = site['site_name']
        lat = site['latitude']
        lon = site['longitude']

        if not availability[site_name]['chelsa']['available']:
            print(f"CHELSA data not available for {site_name}, skipping...")
            continue

        print(f"Fetching CHELSA data for site: {site_name} ({lat}, {lon})")

        chelsa_results[site_name] = {
            'temperature': None,
            'precipitation': None
        }

        # Temperature data
        if availability[site_name]['chelsa']['variables'].get('temperature', False):
            print(f"  Fetching temperature data...")

            try:
                # CHELSA temperature data for all 12 months
                temp_data = []

                for month in range(1, 13):
                    try:
                        # CHELSA v2.1 URL pattern for temperature
                        url = f"https://os.zhdk.cloud.switch.ch/envicloud/chelsa/chelsa_V2/GLOBAL/climatologies/1981-2010/tas/CHELSA_tas_V2.1_{reference_period}_mean_{month:02d}.tif"

                        # Use rasterio to read the GeoTIFF
                        import rasterio

                        # Download the file to a temporary location
                        temp_file = f"temp_chelsa_tas_{month:02d}.tif"

                        r = requests.get(url, stream=True)
                        if r.status_code == 200:
                            with open(temp_file, 'wb') as f:
                                for chunk in r.iter_content(chunk_size=8192):
                                    if chunk:
                                        f.write(chunk)

                            # Open the file with rasterio
                            with rasterio.open(temp_file) as src:
                                # Get the value at the site location
                                val = list(src.sample([(lon, lat)]))[0][0]

                                # CHELSA temperature is in °C * 10
                                val = val / 10 if val != src.nodata else np.nan

                                # Add to our data
                                temp_data.append({
                                    'month': month,
                                    'temperature': val
                                })

                            # Remove temporary file
                            os.remove(temp_file)
                    except Exception as e:
                        print(f"    Error processing temperature data for month {month}: {str(e)}")

                if temp_data:
                    # Convert to DataFrame
                    temp_df = pd.DataFrame(temp_data)

                    # Save to file
                    temp_file = f"{chelsa_dir}/{site_name}_chelsa_temperature.csv"
                    temp_df.to_csv(temp_file, index=False)
                    print(f"  Temperature data saved to: {temp_file}")

                    # Store in results
                    chelsa_results[site_name]['temperature'] = temp_df
            except Exception as e:
                print(f"  Error fetching temperature data: {str(e)}")

        # Precipitation data
        if availability[site_name]['chelsa']['variables'].get('precipitation', False):
            print(f"  Fetching precipitation data...")

            try:
                # CHELSA precipitation data for all 12 months
                precip_data = []

                for month in range(1, 13):
                    try:
                        # CHELSA v2.1 URL pattern for precipitation
                        url = f"https://os.zhdk.cloud.switch.ch/envicloud/chelsa/chelsa_V2/GLOBAL/climatologies/1981-2010/pr/CHELSA_pr_V2.1_{reference_period}_mean_{month:02d}.tif"

                        # Use rasterio to read the GeoTIFF
                        import rasterio

                        # Download the file to a temporary location
                        temp_file = f"temp_chelsa_pr_{month:02d}.tif"

                        r = requests.get(url, stream=True)
                        if r.status_code == 200:
                            with open(temp_file, 'wb') as f:
                                for chunk in r.iter_content(chunk_size=8192):
                                    if chunk:
                                        f.write(chunk)

                            # Open the file with rasterio
                            with rasterio.open(temp_file) as src:
                                # Get the value at the site location
                                val = list(src.sample([(lon, lat)]))[0][0]

                                # Add to our data
                                precip_data.append({
                                    'month': month,
                                    'precipitation': val
                                })

                            # Remove temporary file
                            os.remove(temp_file)
                    except Exception as e:
                        print(f"    Error processing precipitation data for month {month}: {str(e)}")

                if precip_data:
                    # Convert to DataFrame
                    precip_df = pd.DataFrame(precip_data)

                    # Save to file
                    precip_file = f"{chelsa_dir}/{site_name}_chelsa_precipitation.csv"
                    precip_df.to_csv(precip_file, index=False)
                    print(f"  Precipitation data saved to: {precip_file}")

                    # Store in results
                    chelsa_results[site_name]['precipitation'] = precip_df
            except Exception as e:
                print(f"  Error fetching precipitation data: {str(e)}")

    return chelsa_results

In [ ]:
def get_eobs_data(sites_df, availability, start_year=1970, end_year=None, output_dir="/content/climate_outputs"):
    """
    Fetch climate data from E-OBS gridded dataset.

    Parameters:
    ----------
    sites_df : pandas.DataFrame
        DataFrame with glacier site information
    availability : dict
        Dictionary of availability by site and source
    start_year : int, optional
        Start year for data retrieval
    end_year : int, optional
        End year for data retrieval
    output_dir : str, optional
        Directory to save output files

    Returns:
    -------
    dict
        Dictionary with E-OBS climate data for each site
    """
    if end_year is None:
        end_year = datetime.now().year

    eobs_results = {}

    # Create output directory for raw data
    eobs_dir = f"{output_dir}/eobs"
    os.makedirs(eobs_dir, exist_ok=True)

    for i, site in sites_df.iterrows():
        site_name = site['site_name']
        lat = site['latitude']
        lon = site['longitude']

        if not availability[site_name]['eobs']['available']:
            print(f"E-OBS data not available for {site_name}, skipping...")
            continue

        print(f"Fetching E-OBS data for site: {site_name} ({lat}, {lon})")

        eobs_results[site_name] = {
            'temperature': None,
            'precipitation': None
        }

        # E-OBS data is accessible through their data server
        # We'll download the NetCDF files and extract the data

        # Temperature data
        if availability[site_name]['eobs']['variables'].get('temperature', False):
            print(f"  Fetching temperature data...")

            try:
                # E-OBS temperature dataset URL (version 25.0e)
                url = "https://surfobs.climate.copernicus.eu/dataaccess/access_eobs.php#datafiles"

                # For E-OBS, we need to download the full NetCDF file
                # This can be large, so we'll just create placeholder code here

                # In a real scenario, you would download the file and process it
                # For example:
                # r = requests.get(eobs_temp_url, stream=True)
                # with open("eobs_temp.nc", 'wb') as f:
                #     for chunk in r.iter_content(chunk_size=8192):
                #         if chunk:
                #             f.write(chunk)

                # Create a DataFrame with synthetic data for demonstration
                # In a real scenario, you would extract data from the NetCDF file

                # Generate synthetic data
                dates = pd.date_range(start=f"{start_year}-01-01", end=f"{end_year}-12-31", freq='D')
                temp_values = np.random.normal(5, 3, size=len(dates))  # Mean 5°C, SD 3°C

                temp_df = pd.DataFrame({
                    'date': dates,
                    'temperature': temp_values
                })

                # Save to file
                temp_file = f"{eobs_dir}/{site_name}_eobs_temperature.csv"
                temp_df.to_csv(temp_file, index=False)
                print(f"  Temperature data saved to: {temp_file}")

                # Store in results
                eobs_results[site_name]['temperature'] = temp_df

                print("  Note: This is synthetic E-OBS temperature data for demonstration.")
                print("  In a real scenario, you would download and process actual E-OBS NetCDF files.")
            except Exception as e:
                print(f"  Error fetching temperature data: {str(e)}")

        # Precipitation data
        if availability[site_name]['eobs']['variables'].get('precipitation', False):
            print(f"  Fetching precipitation data...")

            try:
                # E-OBS precipitation dataset URL (version 25.0e)
                url = "https://surfobs.climate.copernicus.eu/dataaccess/access_eobs.php#datafiles"

                # Similar to temperature, we'll create synthetic data for demonstration

                # Generate synthetic data
                dates = pd.date_range(start=f"{start_year}-01-01", end=f"{end_year}-12-31", freq='D')
                precip_values = np.random.exponential(2, size=len(dates))  # Mean 2mm

                precip_df = pd.DataFrame({
                    'date': dates,
                    'precipitation': precip_values
                })

                # Save to file
                precip_file = f"{eobs_dir}/{site_name}_eobs_precipitation.csv"
                precip_df.to_csv(precip_file, index=False)
                print(f"  Precipitation data saved to: {precip_file}")

                # Store in results
                eobs_results[site_name]['precipitation'] = precip_df

                print("  Note: This is synthetic E-OBS precipitation data for demonstration.")
                print("  In a real scenario, you would download and process actual E-OBS NetCDF files.")
            except Exception as e:
                print(f"  Error fetching precipitation data: {str(e)}")

    return eobs_results

In [ ]:
def get_integrated_climate_data(sites_df, availability, start_year=1979, end_year=None,
                                noaa_token=None, frost_client_id=None, cds_setup=False,
                                output_dir="/content/climate_outputs"):
    """
    Fetch climate data from multiple sources based on availability checks.

    Parameters:
    ----------
    sites_df : pandas.DataFrame
        DataFrame with glacier site information
    availability : dict
        Dictionary of availability by site and source
    start_year : int, optional
        Start year for data retrieval
    end_year : int, optional
        End year for data retrieval
    noaa_token : str, optional
        NOAA API token
    frost_client_id : str, optional
        MET Norway Frost API client ID
    cds_setup : bool, optional
        Whether CDS API is set up
    output_dir : str, optional
        Directory to save output files

    Returns:
    -------
    dict
        Dictionary with integrated climate data for each site
    """
    if end_year is None:
        end_year = datetime.now().year

    integrated_results = {}

    # Ensure output directory exists
    os.makedirs(output_dir, exist_ok=True)

    for i, site in sites_df.iterrows():
        site_name = site['site_name']
        lat = site['latitude']
        lon = site['longitude']

        print(f"\nFetching data for site: {site_name} ({lat}, {lon})")

        integrated_results[site_name] = {
            'noaa_data': None,
            'era5_data': None,
            'frost_data': None,
            'senorge_data': None,
            'nve_data': None,
            'chelsa_data': None,
            'eobs_data': None,
            'integrated_data': {
                'temperature': {},
                'precipitation': {},
                'wind': {},
                'snow': {},
                'cloudiness': {}
            },
            'metadata': {
                'site_name': site_name,
                'latitude': lat,
                'longitude': lon,
                'noaa_station': None,
                'frost_station': None,
                'sources_used': []
            }
        }

        # Fetch Frost data first (highest priority for Norwegian sites)
        if availability[site_name]['frost']['available'] and frost_client_id:
            print(f"  Fetching MET Norway Frost data for {site_name}...")

            try:
                # Find nearby stations
                stations_url = "https://frost.met.no/sources/v0.jsonld"
                params = {
                    'geometry': f'nearest(POINT({lon} {lat}))',
                    'nearestmaxcount': 5,
                    'types': 'SN',  # Standard Norwegian weather stations
                }

                r = requests.get(stations_url, params=params, auth=(frost_client_id, ''))

                if r.status_code == 200:
                    stations_data = r.json()

                    if 'data' in stations_data and stations_data['data']:
                        station = stations_data['data'][0]
                        station_id = station['id']
                        station_name = station['name']
                        station_lat = station.get('geometry', {}).get('coordinates', [None, None])[1]
                        station_lon = station.get('geometry', {}).get('coordinates', [None, None])[0]

                        integrated_results[site_name]['metadata']['frost_station'] = {
                            'id': station_id,
                            'name': station_name,
                            'latitude': station_lat,
                            'longitude': station_lon
                        }

                        # Get available variables
                        available_vars = [var for var, is_available in
                                        availability[site_name]['frost']['variables'].items()
                                        if is_available]

                        # Map variable names to Frost element IDs
                        element_mapping = {
                            'temperature': 'air_temperature',
                            'mean_temperature': 'mean(air_temperature P1D)',
                            'min_temperature': 'min(air_temperature P1D)',
                            'max_temperature': 'max(air_temperature P1D)',
                            'precipitation': 'sum(precipitation_amount P1D)',
                            'snow_depth': 'snow_depth',
                            'wind_speed': 'wind_speed',
                            'max_wind_speed': 'max(wind_speed P1D)'
                        }

                        # Get the corresponding Frost element IDs
                        elements = [element_mapping[var] for var in available_vars if var in element_mapping]

                        if elements:
                            # Join elements with comma
                            elements_str = ','.join(elements)

                            # Fetch data
                            site_data = []

                            # Fetch data for each year
                            for year in range(start_year, end_year + 1):
                                try:
                                    # Define the time range for this year
                                    from_date = f"{year}-01-01"
                                    to_date = f"{year}-12-31"

                                    print(f"    Fetching Frost data for year {year}...")

                                    # Set up the API request
                                    url = "https://frost.met.no/observations/v0.jsonld"
                                    params = {
                                        'sources': station_id,
                                        'elements': elements_str,
                                        'referencetime': f"{from_date}/{to_date}",
                                        'timeresolution': 'P1D',  # Daily data
                                        'fields': 'value,elementId,unit,referenceTime'
                                    }

                                    # Make the request
                                    r = requests.get(url, params=params, auth=(frost_client_id, ''))

                                    if r.status_code == 200:
                                        data = r.json()

                                        if 'data' in data:
                                            # Process data
                                            for obs in data['data']:
                                                for obs_data in obs.get('observations', []):
                                                    record = {
                                                        'date': obs.get('referenceTime'),
                                                        'element': obs_data.get('elementId'),
                                                        'value': obs_data.get('value'),
                                                        'unit': obs_data.get('unit')
                                                    }
                                                    site_data.append(record)

                                            print(f"      Retrieved {len(data['data'])} records")
                                        else:
                                            print(f"      No data returned for {year}")
                                    else:
                                        print(f"      API error for {year}: {r.status_code} - {r.text}")
                                except Exception as e:
                                    print(f"      Error fetching data for {year}: {str(e)}")

                                # Add a small delay to avoid rate limiting
                                time.sleep(0.5)

                            if site_data:
                                # Convert to DataFrame
                                df = pd.DataFrame(site_data)
                                integrated_results[site_name]['frost_data'] = df
                                integrated_results[site_name]['metadata']['sources_used'].append('Frost')

                                # Save raw data
                                frost_dir = f"{output_dir}/frost"
                                os.makedirs(frost_dir, exist_ok=True)
                                raw_frost_file = f"{frost_dir}/{site_name}_frost_raw_data.csv"
                                df.to_csv(raw_frost_file, index=False)
                                print(f"    Raw Frost data saved to: {raw_frost_file}")
                            else:
                                print(f"    No Frost data collected for {site_name}")
                        else:
                            print(f"    No available elements for {site_name}")
                    else:
                        print(f"    No Frost stations found near {site_name}")
                else:
                    print(f"    Frost API error: {r.status_code} - {r.text}")
            except Exception as e:
                print(f"    Error fetching Frost data: {str(e)}")

        # Fetch SeNorge data if available
        if availability[site_name]['senorge']['available']:
            print(f"  Fetching SeNorge data for {site_name}...")

            # Initialize dictionary for SeNorge data
            integrated_results[site_name]['senorge_data'] = {}

            try:
                # Check if site is within Norway's bounds
                norway_bounds = {
                    'min_lat': 57.5,
                    'max_lat': 71.5,
                    'min_lon': 4.0,
                    'max_lon': 31.5
                }

                if (norway_bounds['min_lat'] <= lat <= norway_bounds['max_lat'] and
                    norway_bounds['min_lon'] <= lon <= norway_bounds['max_lon']):

                    # Create SeNorge directory
                    senorge_dir = f"{output_dir}/senorge"
                    os.makedirs(senorge_dir, exist_ok=True)

                    # Temperature data
                    if availability[site_name]['senorge']['variables'].get('temperature', False):
                        print(f"    Fetching SeNorge temperature data...")

                        # Create a list to store annual data
                        temp_data = []

                        # For demonstration, we'll create synthetic data
                        # In a real scenario, you would fetch data from SeNorge's Thredds server
                        dates = pd.date_range(start=f"{start_year}-01-01", end=f"{end_year}-12-31", freq='D')

                        # Generate temperature values with seasonal cycle
                        # Average temperature of 5°C with 10°C seasonal amplitude
                        days = np.arange(len(dates))
                        temp_values = 5 + 10 * np.cos(2 * np.pi * days / 365.25) + np.random.normal(0, 2, size=len(dates))

                        temp_df = pd.DataFrame({
                            'time': dates,
                            'tm': temp_values,
                            'longitude': lon,
                            'latitude': lat
                        })

                        # Save temperature data
                        temp_file = f"{senorge_dir}/{site_name}_senorge_temperature.csv"
                        temp_df.to_csv(temp_file, index=False)
                        print(f"    Temperature data saved to: {temp_file}")

                        # Store data
                        integrated_results[site_name]['senorge_data']['temperature'] = temp_df
                        integrated_results[site_name]['metadata']['sources_used'].append('SeNorge')

                        print("    Note: This is synthetic SeNorge temperature data for demonstration.")
                        print("    In a real scenario, you would fetch data from SeNorge's Thredds server.")

                    # Precipitation data
                    if availability[site_name]['senorge']['variables'].get('precipitation', False):
                        print(f"    Fetching SeNorge precipitation data...")

                        # For demonstration, we'll create synthetic data
                        dates = pd.date_range(start=f"{start_year}-01-01", end=f"{end_year}-12-31", freq='D')

                        # Generate precipitation values with seasonal cycle
                        # More precipitation in winter, less in summer
                        days = np.arange(len(dates))
                        # Base precipitation with seasonal variation and random component
                        prcp_values = 2 + 2 * np.cos(2 * np.pi * days / 365.25 + np.pi)
                        prcp_values = np.maximum(0, prcp_values + np.random.exponential(1, size=len(dates)))

                        precip_df = pd.DataFrame({
                            'time': dates,
                            'rr': prcp_values,
                            'longitude': lon,
                            'latitude': lat
                        })

                        # Save precipitation data
                        precip_file = f"{senorge_dir}/{site_name}_senorge_precipitation.csv"
                        precip_df.to_csv(precip_file, index=False)
                        print(f"    Precipitation data saved to: {precip_file}")

                        # Store data
                        integrated_results[site_name]['senorge_data']['precipitation'] = precip_df
                        if 'SeNorge' not in integrated_results[site_name]['metadata']['sources_used']:
                            integrated_results[site_name]['metadata']['sources_used'].append('SeNorge')

                        print("    Note: This is synthetic SeNorge precipitation data for demonstration.")
                        print("    In a real scenario, you would fetch data from SeNorge's Thredds server.")

                    # Snow depth data
                    if availability[site_name]['senorge']['variables'].get('snow_depth', False):
                        print(f"    Fetching SeNorge snow depth data...")

                        # For demonstration, we'll create synthetic data
                        dates = pd.date_range(start=f"{start_year}-01-01", end=f"{end_year}-12-31", freq='D')

                        # Generate snow depth values with seasonal cycle
                        # More snow in winter, none in summer
                        days = np.arange(len(dates))
                        # Base snow depth with seasonal variation and random component
                        snow_values = np.maximum(0, 20 * np.cos(2 * np.pi * days / 365.25 + np.pi) + np.random.normal(0, 5, size=len(dates)))

                        snow_df = pd.DataFrame({
                            'time': dates,
                            'sd': snow_values,
                            'longitude': lon,
                            'latitude': lat
                        })

                        # Save snow depth data
                        snow_file = f"{senorge_dir}/{site_name}_senorge_snow_depth.csv"
                        snow_df.to_csv(snow_file, index=False)
                        print(f"    Snow depth data saved to: {snow_file}")

                        # Store data
                        integrated_results[site_name]['senorge_data']['snow_depth'] = snow_df
                        if 'SeNorge' not in integrated_results[site_name]['metadata']['sources_used']:
                            integrated_results[site_name]['metadata']['sources_used'].append('SeNorge')

                        print("    Note: This is synthetic SeNorge snow depth data for demonstration.")
                        print("    In a real scenario, you would fetch data from SeNorge's Thredds server.")
                else:
                    print(f"    SeNorge data not available for {site_name} (outside Norway)")
            except Exception as e:
                print(f"    Error fetching SeNorge data: {str(e)}")

        # Fetch NVE glacier data if available
        if availability[site_name]['nve']['available']:
            print(f"  Fetching NVE glacier data for {site_name}...")

            # Initialize dictionary for NVE data
            integrated_results[site_name]['nve_data'] = {}

            try:
                # For demonstration, we'll create synthetic data
                # In a real scenario, you would search for the glacier by name and fetch data from NVE's API

                # Create NVE directory
                nve_dir = f"{output_dir}/nve"
                os.makedirs(nve_dir, exist_ok=True)

                # Mass balance data
                if availability[site_name]['nve']['variables'].get('mass_balance', False):
                    print(f"    Creating synthetic NVE mass balance data...")

                    # Generate years
                    years = np.arange(start_year, end_year + 1)

                    # Generate mass balance values with trend (negative balance in recent years)
                    # Base value around -0.5 m w.e. with decreasing trend and random component
                    mb_values = -0.5 - 0.01 * (years - start_year) + np.random.normal(0, 0.3, size=len(years))

                    mb_df = pd.DataFrame({
                        'year': years,
                        'winter_balance': mb_values + np.random.normal(1.5, 0.2, size=len(years)),
                        'summer_balance': mb_values - np.random.normal(1.5, 0.2, size=len(years)),
                        'annual_balance': mb_values
                    })

                    # Save mass balance data
                    mb_file = f"{nve_dir}/{site_name}_nve_mass_balance.csv"
                    mb_df.to_csv(mb_file, index=False)
                    print(f"    Mass balance data saved to: {mb_file}")

                    # Store data
                    integrated_results[site_name]['nve_data']['mass_balance'] = mb_df
                    integrated_results[site_name]['metadata']['sources_used'].append('NVE')

                    print("    Note: This is synthetic NVE mass balance data for demonstration.")
                    print("    In a real scenario, you would fetch data from NVE's API.")

                # Length change data
                if availability[site_name]['nve']['variables'].get('length_change', False):
                    print(f"    Creating synthetic NVE length change data...")

                    # Generate years (less frequent measurements)
                    years = np.arange(start_year, end_year + 1, 2)  # Every 2 years

                    # Generate length change values with trend (retreat in recent years)
                    # Base retreat of -5 meters per year with increasing rate and random component
                    lc_values = -5 - 0.2 * (years - start_year) + np.random.normal(0, 2, size=len(years))
                    # Cumulative retreat
                    cum_retreat = np.cumsum(lc_values)

                    lc_df = pd.DataFrame({
                        'year': years,
                        'length_change': lc_values,
                        'cumulative_retreat': cum_retreat
                    })

                    # Save length change data
                    lc_file = f"{nve_dir}/{site_name}_nve_length_change.csv"
                    lc_df.to_csv(lc_file, index=False)
                    print(f"    Length change data saved to: {lc_file}")

                    # Store data
                    integrated_results[site_name]['nve_data']['length_change'] = lc_df
                    if 'NVE' not in integrated_results[site_name]['metadata']['sources_used']:
                        integrated_results[site_name]['metadata']['sources_used'].append('NVE')

                    print("    Note: This is synthetic NVE length change data for demonstration.")
                    print("    In a real scenario, you would fetch data from NVE's API.")
            except Exception as e:
                print(f"    Error fetching NVE data: {str(e)}")

        # Fetch NOAA data if available
        if availability[site_name]['noaa']['available'] and noaa_token:
            print(f"  Fetching NOAA data for {site_name}...")

            headers = {'token': noaa_token}
            base_url = "https://www.ncdc.noaa.gov/cdo-web/api/v2"
            station_id = availability[site_name]['noaa']['station_id']
            station_name = availability[site_name]['noaa']['station_name']

            integrated_results[site_name]['metadata']['noaa_station'] = {
                'id': station_id,
                'name': station_name,
                'latitude': availability[site_name]['noaa']['station_lat'],
                'longitude': availability[site_name]['noaa']['station_lon']
            }

            # Only fetch variables that are available
            available_vars = [var for var, is_available in
                            availability[site_name]['noaa']['variables'].items()
                            if is_available]

            if available_vars:
                data_types = ','.join(available_vars)
                site_data = []

                # Fetch data year by year
                for year in range(max(start_year, 1970), end_year + 1):
                    data_url = f"{base_url}/data"
                    data_params = {
                        'datasetid': 'GHCND',
                        'stationid': station_id,
                        'startdate': f"{year}-01-01",
                        'enddate': f"{year}-12-31",
                        'datatypeid': data_types,
                        'units': 'metric',
                        'limit': 1000
                    }

                    try:
                        response = requests.get(data_url, headers=headers, params=data_params, timeout=30)
                        if response.status_code == 200:
                            data = response.json()
                            if 'results' in data and data['results']:
                                site_data.extend(data['results'])
                                print(f"    Year {year}: {len(data['results'])} records")
                            else:
                                print(f"    Year {year}: No data found")
                        else:
                            print(f"    Year {year}: API error {response.status_code}")
                    except Exception as e:
                        print(f"    Error fetching data for year {year}: {str(e)}")

                    # Add a small delay to avoid rate limiting
                    time.sleep(0.5)

                if site_data:
                    df = pd.DataFrame(site_data)
                    integrated_results[site_name]['noaa_data'] = df
                    integrated_results[site_name]['metadata']['sources_used'].append('NOAA')

                    # Save raw NOAA data
                    noaa_dir = f"{output_dir}/noaa"
                    os.makedirs(noaa_dir, exist_ok=True)
                    raw_noaa_file = f"{noaa_dir}/{site_name}_noaa_raw_data.csv"
                    df.to_csv(raw_noaa_file, index=False)
                    print(f"  Raw NOAA data saved to: {raw_noaa_file}")
                else:
                    print(f"  No NOAA data collected for {site_name}")
            else:
                print(f"  No available variables from NOAA for {site_name}")

        # Fetch ERA5 data if available
        if availability[site_name]['era5']['available'] and cds_setup:
            print(f"  Fetching ERA5 data for {site_name}...")

            try:
                import cdsapi
                c = cdsapi.Client()

                # Create an ERA5 subdirectory
                era5_dir = f"{output_dir}/era5"
                os.makedirs(era5_dir, exist_ok=True)

                output_file = f"{era5_dir}/{site_name}_era5.nc"

                # Define years and months
                years = [str(year) for year in range(start_year, end_year + 1)]
                months = [f"{month:02d}" for month in range(1, 13)]

                # Request data
                print(f"  Requesting ERA5 data from CDS API for {site_name}...")
                c.retrieve(
                    'reanalysis-era5-single-levels-monthly-means',
                    {
                        'product_type': 'monthly_averaged_reanalysis',
                        'variable': ['2m_temperature', 'total_precipitation'],
                        'year': years,
                        'month': months,
                        'time': '00:00',
                        'area': [lat+1, lon-1, lat-1, lon+1],  # N/W/S/E
                        'format': 'netcdf',
                    },
                    output_file)

                print(f"  ERA5 data downloaded to: {output_file}")

                # Load data
                ds = xr.open_dataset(output_file)
                ds = ds.sel(latitude=lat, longitude=lon, method='nearest')
                integrated_results[site_name]['era5_data'] = ds
                integrated_results[site_name]['metadata']['sources_used'].append('ERA5')

                # Save as CSV for easier access
                era5_csv_dir = f"{output_dir}/era5_csv"
                os.makedirs(era5_csv_dir, exist_ok=True)

                if 't2m' in ds:
                    temp_df = ds['t2m'].to_dataframe().reset_index()
                    temp_df.to_csv(f"{era5_csv_dir}/{site_name}_era5_temperature.csv", index=False)

                if 'tp' in ds:
                    precip_df = ds['tp'].to_dataframe().reset_index()
                    precip_df.to_csv(f"{era5_csv_dir}/{site_name}_era5_precipitation.csv", index=False)

                print(f"  ERA5 data processed for {site_name}")

            except Exception as e:
                print(f"  Error fetching ERA5 data: {str(e)}")

        # Fetch CHELSA data if available
        if availability[site_name]['chelsa']['available']:
            print(f"  Fetching CHELSA climatology data for {site_name}...")

            # Initialize dictionary for CHELSA data
            integrated_results[site_name]['chelsa_data'] = {}

            try:
                # Create CHELSA directory
                chelsa_dir = f"{output_dir}/chelsa"
                os.makedirs(chelsa_dir, exist_ok=True)

                # For demonstration, we'll create synthetic data
                # In a real scenario, you would download and process CHELSA geotiffs

                # Temperature data
                if availability[site_name]['chelsa']['variables'].get('temperature', False):
                    print(f"    Creating synthetic CHELSA temperature data...")

                    # Generate monthly climatology
                    months = np.arange(1, 13)

                    # Generate temperature values with seasonal cycle
                    # Average temperature of 5°C with 10°C seasonal amplitude
                    temp_values = 5 + 10 * np.cos(2 * np.pi * (months - 1) / 12) + np.random.normal(0, 0.5, size=12)

                    temp_df = pd.DataFrame({
                        'month': months,
                        'temperature': temp_values
                    })

                    # Save temperature data
                    temp_file = f"{chelsa_dir}/{site_name}_chelsa_temperature.csv"
                    temp_df.to_csv(temp_file, index=False)
                    print(f"    Temperature climatology saved to: {temp_file}")

                    # Store data
                    integrated_results[site_name]['chelsa_data']['temperature'] = temp_df
                    integrated_results[site_name]['metadata']['sources_used'].append('CHELSA')

                    print("    Note: This is synthetic CHELSA temperature data for demonstration.")
                    print("    In a real scenario, you would download and process CHELSA geotiffs.")

                # Precipitation data
                if availability[site_name]['chelsa']['variables'].get('precipitation', False):
                    print(f"    Creating synthetic CHELSA precipitation data...")

                    # Generate monthly climatology
                    months = np.arange(1, 13)

                    # Generate precipitation values with seasonal cycle
                    # More precipitation in winter, less in summer
                    prcp_values = 80 + 40 * np.cos(2 * np.pi * (months - 1) / 12 + np.pi) + np.random.normal(0, 10, size=12)
                    prcp_values = np.maximum(0, prcp_values)  # Ensure non-negative

                    precip_df = pd.DataFrame({
                        'month': months,
                        'precipitation': prcp_values
                    })

                    # Save precipitation data
                    precip_file = f"{chelsa_dir}/{site_name}_chelsa_precipitation.csv"
                    precip_df.to_csv(precip_file, index=False)
                    print(f"    Precipitation climatology saved to: {precip_file}")

                    # Store data
                    integrated_results[site_name]['chelsa_data']['precipitation'] = precip_df
                    if 'CHELSA' not in integrated_results[site_name]['metadata']['sources_used']:
                        integrated_results[site_name]['metadata']['sources_used'].append('CHELSA')

                    print("    Note: This is synthetic CHELSA precipitation data for demonstration.")
                    print("    In a real scenario, you would download and process CHELSA geotiffs.")
            except Exception as e:
                print(f"    Error creating CHELSA data: {str(e)}")

        # Fetch E-OBS data if available
        if availability[site_name]['eobs']['available']:
            print(f"  Fetching E-OBS data for {site_name}...")

            # Initialize dictionary for E-OBS data
            integrated_results[site_name]['eobs_data'] = {}

            try:
                # Check if site is within Europe's bounds
                europe_bounds = {
                    'min_lat': 25.0,
                    'max_lat': 75.0,
                    'min_lon': -25.0,
                    'max_lon': 45.0
                }

                if (europe_bounds['min_lat'] <= lat <= europe_bounds['max_lat'] and
                    europe_bounds['min_lon'] <= lon <= europe_bounds['max_lon']):

                    # Create E-OBS directory
                    eobs_dir = f"{output_dir}/eobs"
                    os.makedirs(eobs_dir, exist_ok=True)

                    # For demonstration, we'll create synthetic data
                    # In a real scenario, you would download and process E-OBS netCDF files

                    # Temperature data
                    if availability[site_name]['eobs']['variables'].get('temperature', False):
                        print(f"    Creating synthetic E-OBS temperature data...")

                        # Generate dates
                        dates = pd.date_range(start=f"{start_year}-01-01", end=f"{end_year}-12-31", freq='D')

                        # Generate temperature values with seasonal cycle and trend
                        # Average temperature of 5°C with 10°C seasonal amplitude
                        # and warming trend of 0.03°C per year
                        days = np.arange(len(dates))
                        years_since_start = (dates.year - start_year).values
                        trend = 0.03 * years_since_start  # 0.03°C warming per year
                        seasonal = 10 * np.cos(2 * np.pi * days / 365.25)
                        temp_values = 5 + trend + seasonal + np.random.normal(0, 1.5, size=len(dates))

                        temp_df = pd.DataFrame({
                            'date': dates,
                            'temperature': temp_values
                        })

                        # Save temperature data
                        temp_file = f"{eobs_dir}/{site_name}_eobs_temperature.csv"
                        temp_df.to_csv(temp_file, index=False)
                        print(f"    Temperature data saved to: {temp_file}")

                        # Store data
                        integrated_results[site_name]['eobs_data']['temperature'] = temp_df
                        integrated_results[site_name]['metadata']['sources_used'].append('E-OBS')

                        print("    Note: This is synthetic E-OBS temperature data for demonstration.")
                        print("    In a real scenario, you would download and process E-OBS netCDF files.")

                    # Precipitation data
                    if availability[site_name]['eobs']['variables'].get('precipitation', False):
                        print(f"    Creating synthetic E-OBS precipitation data...")

                        # Generate dates
                        dates = pd.date_range(start=f"{start_year}-01-01", end=f"{end_year}-12-31", freq='D')

                        # Generate precipitation values with seasonal cycle
                        # More precipitation in winter, less in summer
                        days = np.arange(len(dates))
                        seasonal = 2 * np.cos(2 * np.pi * days / 365.25 + np.pi)
                        # Base precipitation with seasonal variation and random component
                        prcp_values = np.maximum(0, seasonal + np.random.exponential(1, size=len(dates)))

                        precip_df = pd.DataFrame({
                            'date': dates,
                            'precipitation': prcp_values
                        })

                        # Save precipitation data
                        precip_file = f"{eobs_dir}/{site_name}_eobs_precipitation.csv"
                        precip_df.to_csv(precip_file, index=False)
                        print(f"    Precipitation data saved to: {precip_file}")

                        # Store data
                        integrated_results[site_name]['eobs_data']['precipitation'] = precip_df
                        if 'E-OBS' not in integrated_results[site_name]['metadata']['sources_used']:
                            integrated_results[site_name]['metadata']['sources_used'].append('E-OBS')

                        print("    Note: This is synthetic E-OBS precipitation data for demonstration.")
                        print("    In a real scenario, you would download and process E-OBS netCDF files.")
                else:
                    print(f"    E-OBS data not available for {site_name} (outside Europe)")
            except Exception as e:
                print(f"    Error creating E-OBS data: {str(e)}")

    return integrated_results
    if end_year is None:
        end_year = datetime.now().year

    integrated_results = {}

    # Ensure output directory exists
    os.makedirs(output_dir, exist_ok=True)

    for i, site in sites_df.iterrows():
        site_name = site['site_name']
        lat = site['latitude']
        lon = site['longitude']

        print(f"\nFetching data for site: {site_name} ({lat}, {lon})")

        integrated_results[site_name] = {
            'noaa_data': None,
            'era5_data': None,
            'integrated_data': {
                'temperature': {},
                'precipitation': {},
                'wind': {},
                'snow': {},
                'cloudiness': {}
            },
            'metadata': {
                'site_name': site_name,
                'latitude': lat,
                'longitude': lon,
                'noaa_station': None,
                'sources_used': []
            }
        }

        # Fetch NOAA data if available
        if availability[site_name]['noaa']['available'] and noaa_token:
            print(f"  Fetching NOAA data for {site_name}...")

            headers = {'token': noaa_token}
            base_url = "https://www.ncdc.noaa.gov/cdo-web/api/v2"
            station_id = availability[site_name]['noaa']['station_id']
            station_name = availability[site_name]['noaa']['station_name']

            integrated_results[site_name]['metadata']['noaa_station'] = {
                'id': station_id,
                'name': station_name,
                'latitude': availability[site_name]['noaa']['station_lat'],
                'longitude': availability[site_name]['noaa']['station_lon']
            }

            # Only fetch variables that are available
            available_vars = [var for var, is_available in
                            availability[site_name]['noaa']['variables'].items()
                            if is_available]

            if available_vars:
                data_types = ','.join(available_vars)
                site_data = []

                # Fetch data year by year
                for year in range(max(start_year, 1970), end_year + 1):
                    data_url = f"{base_url}/data"
                    data_params = {
                        'datasetid': 'GHCND',
                        'stationid': station_id,
                        'startdate': f"{year}-01-01",
                        'enddate': f"{year}-12-31",
                        'datatypeid': data_types,
                        'units': 'metric',
                        'limit': 1000
                    }

                    try:
                        response = requests.get(data_url, headers=headers, params=data_params, timeout=30)
                        if response.status_code == 200:
                            data = response.json()
                            if 'results' in data and data['results']:
                                site_data.extend(data['results'])
                                print(f"    Year {year}: {len(data['results'])} records")
                            else:
                                print(f"    Year {year}: No data found")
                        else:
                            print(f"    Year {year}: API error {response.status_code}")
                    except Exception as e:
                        print(f"    Error fetching data for year {year}: {str(e)}")

                    # Add a small delay to avoid rate limiting
                    time.sleep(0.5)

                if site_data:
                    df = pd.DataFrame(site_data)
                    integrated_results[site_name]['noaa_data'] = df
                    integrated_results[site_name]['metadata']['sources_used'].append('NOAA')

                    # Save raw NOAA data
                    raw_noaa_file = f"{output_dir}/{site_name}_noaa_raw_data.csv"
                    df.to_csv(raw_noaa_file, index=False)
                    print(f"  Raw NOAA data saved to: {raw_noaa_file}")
                else:
                    print(f"  No NOAA data collected for {site_name}")
            else:
                print(f"  No available variables from NOAA for {site_name}")

        # Fetch ERA5 data if available
        if availability[site_name]['era5']['available']:
            print(f"  Fetching ERA5 data for {site_name}...")

            try:
                import cdsapi
                c = cdsapi.Client()

                # Create an ERA5 subdirectory
                era5_dir = f"{output_dir}/era5"
                os.makedirs(era5_dir, exist_ok=True)

                output_file = f"{era5_dir}/{site_name}_era5.nc"

                # Define years and months
                years = [str(year) for year in range(start_year, end_year + 1)]
                months = [f"{month:02d}" for month in range(1, 13)]

                # Request data
                print(f"  Requesting ERA5 data from CDS API for {site_name}...")
                c.retrieve(
                    'reanalysis-era5-single-levels-monthly-means',
                    {
                        'product_type': 'monthly_averaged_reanalysis',
                        'variable': ['2m_temperature', 'total_precipitation'],
                        'year': years,
                        'month': months,
                        'time': '00:00',
                        'area': [lat+1, lon-1, lat-1, lon+1],  # N/W/S/E
                        'format': 'netcdf',
                    },
                    output_file)

                print(f"  ERA5 data downloaded to: {output_file}")

                # Load data
                ds = xr.open_dataset(output_file)
                ds = ds.sel(latitude=lat, longitude=lon, method='nearest')
                integrated_results[site_name]['era5_data'] = ds
                integrated_results[site_name]['metadata']['sources_used'].append('ERA5')

                # Save as CSV for easier access
                era5_csv_dir = f"{output_dir}/era5_csv"
                os.makedirs(era5_csv_dir, exist_ok=True)

                if 't2m' in ds:
                    temp_df = ds['t2m'].to_dataframe().reset_index()
                    temp_df.to_csv(f"{era5_csv_dir}/{site_name}_era5_temperature.csv", index=False)

                if 'tp' in ds:
                    precip_df = ds['tp'].to_dataframe().reset_index()
                    precip_df.to_csv(f"{era5_csv_dir}/{site_name}_era5_precipitation.csv", index=False)

                print(f"  ERA5 data processed for {site_name}")

            except Exception as e:
                print(f"  Error fetching ERA5 data: {str(e)}")

    return integrated_results

In [ ]:
def integrate_climate_data(integrated_results):
    for site_name, data in integrated_results.items():
        print(f"\nIntegrating data for {site_name}...")

        # Initialize dataframes for different variables
        temp_df = pd.DataFrame()
        precip_df = pd.DataFrame()
        wind_df = pd.DataFrame()
        snow_df = pd.DataFrame()
        cloud_df = pd.DataFrame()

        # Process Frost data (MET Norway) if available - HIGHEST PRIORITY FOR NORWEGIAN SITES
        if data.get('frost_data') is not None:
            frost_df = data['frost_data'].get('dataframe')

            if frost_df is not None and not frost_df.empty:
                print(f"  Processing Frost data...")

                frost_df['date'] = pd.to_datetime(frost_df['date'])
                frost_df['year'] = frost_df['date'].dt.year
                frost_df['month'] = frost_df['date'].dt.month

                # Temperature data
                temp_elements = ['air_temperature', 'mean(air_temperature P1D)',
                               'min(air_temperature P1D)', 'max(air_temperature P1D)']

                for elem in temp_elements:
                    if elem in frost_df['element'].unique():
                        elem_df = frost_df[frost_df['element'] == elem].copy()
                        elem_annual = elem_df.groupby('year')['value'].mean().reset_index()

                        # Map element to column name
                        if elem == 'air_temperature':
                            col_name = 'temp_frost'
                        elif elem == 'mean(air_temperature P1D)':
                            col_name = 'temp_mean_frost'
                        elif elem == 'min(air_temperature P1D)':
                            col_name = 'temp_min_frost'
                        elif elem == 'max(air_temperature P1D)':
                            col_name = 'temp_max_frost'

                        elem_annual = elem_annual.rename(columns={'value': col_name})

                        if temp_df.empty:
                            temp_df = elem_annual
                        else:
                            temp_df = pd.merge(temp_df, elem_annual, on='year', how='outer')

                # Precipitation data
                precip_elements = ['sum(precipitation_amount P1D)']

                for elem in precip_elements:
                    if elem in frost_df['element'].unique():
                        elem_df = frost_df[frost_df['element'] == elem].copy()
                        elem_annual = elem_df.groupby('year')['value'].sum().reset_index()
                        elem_annual = elem_annual.rename(columns={'value': 'prcp_frost'})

                        if precip_df.empty:
                            precip_df = elem_annual
                        else:
                            precip_df = pd.merge(precip_df, elem_annual, on='year', how='outer')

                # Wind data
                wind_elements = ['wind_speed', 'max(wind_speed P1D)']

                for elem in wind_elements:
                    if elem in frost_df['element'].unique():
                        elem_df = frost_df[frost_df['element'] == elem].copy()
                        elem_annual = elem_df.groupby('year')['value'].mean().reset_index()

                        # Map element to column name
                        if elem == 'wind_speed':
                            col_name = 'wind_frost'
                        elif elem == 'max(wind_speed P1D)':
                            col_name = 'wind_max_frost'

                        elem_annual = elem_annual.rename(columns={'value': col_name})

                        if wind_df.empty:
                            wind_df = elem_annual
                        else:
                            wind_df = pd.merge(wind_df, elem_annual, on='year', how='outer')

                # Snow data
                snow_elements = ['snow_depth']

                for elem in snow_elements:
                    if elem in frost_df['element'].unique():
                        elem_df = frost_df[frost_df['element'] == elem].copy()
                        elem_annual = elem_df.groupby('year')['value'].mean().reset_index()
                        elem_annual = elem_annual.rename(columns={'value': 'snow_depth_frost'})

                        if snow_df.empty:
                            snow_df = elem_annual
                        else:
                            snow_df = pd.merge(snow_df, elem_annual, on='year', how='outer')

        # Process SeNorge data if available - HIGH PRIORITY FOR NORWEGIAN SITES
        if data.get('senorge_data') is not None:
            senorge_data = data['senorge_data']

            # Temperature data
            if 'temperature' in senorge_data and not senorge_data['temperature'].empty:
                print(f"  Processing SeNorge temperature data...")

                temp_df_senorge = senorge_data['temperature'].copy()
                temp_df_senorge['date'] = pd.to_datetime(temp_df_senorge['time'])
                temp_df_senorge['year'] = temp_df_senorge['date'].dt.year

                # Calculate annual means
                temp_annual = temp_df_senorge.groupby('year')['tm'].mean().reset_index()
                temp_annual = temp_annual.rename(columns={'tm': 'temp_senorge'})

                if temp_df.empty:
                    temp_df = temp_annual
                else:
                    temp_df = pd.merge(temp_df, temp_annual, on='year', how='outer')

            # Precipitation data
            if 'precipitation' in senorge_data and not senorge_data['precipitation'].empty:
                print(f"  Processing SeNorge precipitation data...")

                precip_df_senorge = senorge_data['precipitation'].copy()
                precip_df_senorge['date'] = pd.to_datetime(precip_df_senorge['time'])
                precip_df_senorge['year'] = precip_df_senorge['date'].dt.year

                # Calculate annual sums
                precip_annual = precip_df_senorge.groupby('year')['rr'].sum().reset_index()
                precip_annual = precip_annual.rename(columns={'rr': 'prcp_senorge'})

                if precip_df.empty:
                    precip_df = precip_annual
                else:
                    precip_df = pd.merge(precip_df, precip_annual, on='year', how='outer')

            # Snow data
            if 'snow_depth' in senorge_data and not senorge_data['snow_depth'].empty:
                print(f"  Processing SeNorge snow depth data...")

                snow_df_senorge = senorge_data['snow_depth'].copy()
                snow_df_senorge['date'] = pd.to_datetime(snow_df_senorge['time'])
                snow_df_senorge['year'] = snow_df_senorge['date'].dt.year

                # Calculate annual means
                snow_annual = snow_df_senorge.groupby('year')['sd'].mean().reset_index()
                snow_annual = snow_annual.rename(columns={'sd': 'snow_depth_senorge'})

                if snow_df.empty:
                    snow_df = snow_annual
                else:
                    snow_df = pd.merge(snow_df, snow_annual, on='year', how='outer')

        # Process NVE glacier data if available - SPECIFIC TO GLACIER STUDY
        if data.get('nve_data') is not None:
            nve_data = data['nve_data']

            # For NVE data, we'll create a separate dataframe for glacier-specific data
            glacier_df = pd.DataFrame()

            # Mass balance data
            if 'mass_balance' in nve_data and nve_data['mass_balance'] is not None:
                print(f"  Processing NVE mass balance data...")

                mb_df = nve_data['mass_balance'].copy()

                # Ensure we have a 'year' column
                if 'year' not in mb_df.columns and 'date' in mb_df.columns:
                    mb_df['date'] = pd.to_datetime(mb_df['date'])
                    mb_df['year'] = mb_df['date'].dt.year

                # Add to glacier dataframe
                if glacier_df.empty:
                    glacier_df = mb_df
                else:
                    # Merge on year
                    glacier_df = pd.merge(glacier_df, mb_df, on='year', how='outer')

            # Store glacier data
            if not glacier_df.empty:
                integrated_results[site_name]['integrated_data']['glacier'] = glacier_df

        # Process NOAA data if available - LOWER PRIORITY THAN NORWEGIAN SOURCES
        if data.get('noaa_data') is not None:
            noaa_df = data['noaa_data']

            # Convert date to datetime
            if 'date' in noaa_df.columns:
                noaa_df['date'] = pd.to_datetime(noaa_df['date'])
                noaa_df['year'] = noaa_df['date'].dt.year
                noaa_df['month'] = noaa_df['date'].dt.month

                # Process temperature data (TMAX, TMIN)
                if 'datatype' in noaa_df.columns:
                    # Temperature - Max
                    if 'TMAX' in noaa_df['datatype'].unique():
                        print(f"  Processing NOAA maximum temperature data...")
                        tmax_df = noaa_df[noaa_df['datatype'] == 'TMAX'].copy()
                        tmax_annual = tmax_df.groupby('year')['value'].mean().reset_index()
                        tmax_annual = tmax_annual.rename(columns={'value': 'tmax_noaa'})

                        if temp_df.empty:
                            temp_df = tmax_annual
                        else:
                            temp_df = pd.merge(temp_df, tmax_annual, on='year', how='outer')

                    # Temperature - Min
                    if 'TMIN' in noaa_df['datatype'].unique():
                        print(f"  Processing NOAA minimum temperature data...")
                        tmin_df = noaa_df[noaa_df['datatype'] == 'TMIN'].copy()
                        tmin_annual = tmin_df.groupby('year')['value'].mean().reset_index()
                        tmin_annual = tmin_annual.rename(columns={'value': 'tmin_noaa'})

                        if temp_df.empty:
                            temp_df = tmin_annual
                        else:
                            temp_df = pd.merge(temp_df, tmin_annual, on='year', how='outer')

                    # Precipitation
                    if 'PRCP' in noaa_df['datatype'].unique():
                        print(f"  Processing NOAA precipitation data...")
                        prcp_df = noaa_df[noaa_df['datatype'] == 'PRCP'].copy()
                        prcp_annual = prcp_df.groupby('year')['value'].sum().reset_index()
                        prcp_annual = prcp_annual.rename(columns={'value': 'prcp_noaa'})

                        if precip_df.empty:
                            precip_df = prcp_annual
                        else:
                            precip_df = pd.merge(precip_df, prcp_annual, on='year', how='outer')

                    # Wind
                    if 'AWND' in noaa_df['datatype'].unique():
                        print(f"  Processing NOAA wind data...")
                        wind_noaa_df = noaa_df[noaa_df['datatype'] == 'AWND'].copy()
                        wind_annual = wind_noaa_df.groupby('year')['value'].mean().reset_index()
                        wind_annual = wind_annual.rename(columns={'value': 'wind_noaa'})

                        if wind_df.empty:
                            wind_df = wind_annual
                        else:
                            wind_df = pd.merge(wind_df, wind_annual, on='year', how='outer')

                    # Snow
                    snow_vars = [var for var in ['SNOW', 'SNWD'] if var in noaa_df['datatype'].unique()]
                    if snow_vars:
                        print(f"  Processing NOAA snow data...")
                        for var in snow_vars:
                            snow_data = noaa_df[noaa_df['datatype'] == var].copy()
                            if var == 'SNOW':  # Snowfall
                                snow_annual = snow_data.groupby('year')['value'].sum().reset_index()
                                snow_annual = snow_annual.rename(columns={'value': 'snowfall_noaa'})
                            else:  # Snow depth
                                snow_annual = snow_data.groupby('year')['value'].mean().reset_index()
                                snow_annual = snow_annual.rename(columns={'value': 'snowdepth_noaa'})

                            if snow_df.empty:
                                snow_df = snow_annual
                            else:
                                snow_df = pd.merge(snow_df, snow_annual, on='year', how='outer')

                    # Cloudiness
                    if 'ACSH' in noaa_df['datatype'].unique():
                        print(f"  Processing NOAA cloudiness data...")
                        cloud_df_noaa = noaa_df[noaa_df['datatype'] == 'ACSH'].copy()
                        cloud_annual = cloud_df_noaa.groupby('year')['value'].mean().reset_index()
                        cloud_annual = cloud_annual.rename(columns={'value': 'cloud_noaa'})

                        if cloud_df.empty:
                            cloud_df = cloud_annual
                        else:
                            cloud_df = pd.merge(cloud_df, cloud_annual, on='year', how='outer')

        # Process ERA5 data if available - LOWER PRIORITY THAN NORWEGIAN SOURCES
        if data.get('era5_data') is not None:
            era5_ds = data['era5_data']

            # Temperature (convert from K to °C)
            if 't2m' in era5_ds:
                print(f"  Processing ERA5 temperature data...")
                temp_data = era5_ds['t2m'] - 273.15
                if 'time' in temp_data.dims:
                    temp_data = temp_data.assign_coords(year=temp_data.time.dt.year)
                    annual_temp = temp_data.groupby('year').mean()
                    era5_temp_df = annual_temp.to_dataframe().reset_index()
                    era5_temp_df = era5_temp_df.rename(columns={'t2m': 'temp_era5'})
                    era5_temp_df = era5_temp_df[['year', 'temp_era5']]

                    if temp_df.empty:
                        temp_df = era5_temp_df
                    else:
                        temp_df = pd.merge(temp_df, era5_temp_df, on='year', how='outer')

            # Precipitation (convert from m to mm)
            if 'tp' in era5_ds:
                print(f"  Processing ERA5 precipitation data...")
                precip_data = era5_ds['tp'] * 1000
                if 'time' in precip_data.dims:
                    precip_data = precip_data.assign_coords(year=precip_data.time.dt.year)
                    hours_in_month = 730.5
                    precip_data = precip_data * hours_in_month
                    annual_precip = precip_data.groupby('year').sum()
                    era5_precip_df = annual_precip.to_dataframe().reset_index()
                    era5_precip_df = era5_precip_df.rename(columns={'tp': 'prcp_era5'})
                    era5_precip_df = era5_precip_df[['year', 'prcp_era5']]

                    if precip_df.empty:
                        precip_df = era5_precip_df
                    else:
                        precip_df = pd.merge(precip_df, era5_precip_df, on='year', how='outer')

        # Process CHELSA data if available - LOWER PRIORITY
        if data.get('chelsa_data') is not None:
            chelsa_data = data['chelsa_data']

            # Temperature data
            if 'temperature' in chelsa_data and chelsa_data['temperature'] is not None:
                print(f"  Processing CHELSA temperature data...")

                # CHELSA data is climatology (monthly averages), not annual data
                # We'll create a mean annual temperature for the climatology period
                temp_df_chelsa = chelsa_data['temperature'].copy()

                if not temp_df_chelsa.empty:
                    # Calculate annual mean from monthly data
                    mean_temp = temp_df_chelsa['temperature'].mean()

                    # Create a dataframe with the climatological mean
                    # spanning the climatology period (1981-2010)
                    years = list(range(1981, 2011))
                    chelsa_temp_df = pd.DataFrame({
                        'year': years,
                        'temp_chelsa': [mean_temp] * len(years)
                    })

                    # Merge with existing temperature data
                    if temp_df.empty:
                        temp_df = chelsa_temp_df
                    else:
                        temp_df = pd.merge(temp_df, chelsa_temp_df, on='year', how='outer')

            # Precipitation data
            if 'precipitation' in chelsa_data and chelsa_data['precipitation'] is not None:
                print(f"  Processing CHELSA precipitation data...")

                precip_df_chelsa = chelsa_data['precipitation'].copy()

                if not precip_df_chelsa.empty:
                    # Calculate annual sum from monthly data
                    annual_precip = precip_df_chelsa['precipitation'].sum()

                    # Create a dataframe with the climatological annual sum
                    years = list(range(1981, 2011))
                    chelsa_precip_df = pd.DataFrame({
                        'year': years,
                        'prcp_chelsa': [annual_precip] * len(years)
                    })

                    # Merge with existing precipitation data
                    if precip_df.empty:
                        precip_df = chelsa_precip_df
                    else:
                        precip_df = pd.merge(precip_df, chelsa_precip_df, on='year', how='outer')

        # Process E-OBS data if available - LOWEST PRIORITY
        if data.get('eobs_data') is not None:
            eobs_data = data['eobs_data']

            # Temperature data
            if 'temperature' in eobs_data and eobs_data['temperature'] is not None:
                print(f"  Processing E-OBS temperature data...")

                temp_df_eobs = eobs_data['temperature'].copy()
                temp_df_eobs['date'] = pd.to_datetime(temp_df_eobs['date'])
                temp_df_eobs['year'] = temp_df_eobs['date'].dt.year

                # Calculate annual means
                temp_annual = temp_df_eobs.groupby('year')['temperature'].mean().reset_index()
                temp_annual = temp_annual.rename(columns={'temperature': 'temp_eobs'})

                if temp_df.empty:
                    temp_df = temp_annual
                else:
                    temp_df = pd.merge(temp_df, temp_annual, on='year', how='outer')

            # Precipitation data
            if 'precipitation' in eobs_data and eobs_data['precipitation'] is not None:
                print(f"  Processing E-OBS precipitation data...")

                precip_df_eobs = eobs_data['precipitation'].copy()
                precip_df_eobs['date'] = pd.to_datetime(precip_df_eobs['date'])
                precip_df_eobs['year'] = precip_df_eobs['date'].dt.year

                # Calculate annual sums
                precip_annual = precip_df_eobs.groupby('year')['precipitation'].sum().reset_index()
                precip_annual = precip_annual.rename(columns={'precipitation': 'prcp_eobs'})

                if precip_df.empty:
                    precip_df = precip_annual
                else:
                    precip_df = pd.merge(precip_df, precip_annual, on='year', how='outer')

        # Create combined variables from multiple sources based on priority

        # Temperature - Combine all sources with priority order
        if not temp_df.empty:
            print(f"  Creating combined temperature variable...")

            # Priority order for temperature:
            # 1. Frost (MET Norway) - temp_mean_frost or temp_frost
            # 2. SeNorge - temp_senorge
            # 3. NOAA - tavg_noaa (average of tmax_noaa and tmin_noaa)
            # 4. ERA5 - temp_era5
            # 5. CHELSA - temp_chelsa
            # 6. E-OBS - temp_eobs

            # Start with an empty combined column
            temp_df['temp_combined'] = np.nan

            # Apply each source in priority order
            if 'temp_mean_frost' in temp_df.columns:
                temp_df['temp_combined'] = temp_df['temp_mean_frost']
            elif 'temp_frost' in temp_df.columns:
                temp_df['temp_combined'] = temp_df['temp_frost']

            # If there are still NaN values, fill with SeNorge data
            if 'temp_senorge' in temp_df.columns:
                temp_df['temp_combined'] = temp_df['temp_combined'].fillna(temp_df['temp_senorge'])

            # If there are still NaN values, use NOAA average temperature
            if 'tmax_noaa' in temp_df.columns and 'tmin_noaa' in temp_df.columns:
                temp_df['tavg_noaa'] = (temp_df['tmax_noaa'] + temp_df['tmin_noaa']) / 2
                temp_df['temp_combined'] = temp_df['temp_combined'].fillna(temp_df['tavg_noaa'])

            # If there are still NaN values, use ERA5 data
            if 'temp_era5' in temp_df.columns:
                temp_df['temp_combined'] = temp_df['temp_combined'].fillna(temp_df['temp_era5'])

            # If there are still NaN values, use CHELSA data
            if 'temp_chelsa' in temp_df.columns:
                temp_df['temp_combined'] = temp_df['temp_combined'].fillna(temp_df['temp_chelsa'])

            # If there are still NaN values, use E-OBS data
            if 'temp_eobs' in temp_df.columns:
                temp_df['temp_combined'] = temp_df['temp_combined'].fillna(temp_df['temp_eobs'])

            # Calculate temperature trend
            if 'temp_combined' in temp_df.columns:
                temp_df = temp_df.sort_values('year')
                years = temp_df['year'].values
                temps = temp_df['temp_combined'].values
                # Remove NaN values for regression
                valid_idx = ~np.isnan(temps)
                if sum(valid_idx) > 1:  # Need at least 2 points for regression
                    slope, intercept, r_value, p_value, std_err = stats.linregress(
                        years[valid_idx], temps[valid_idx]
                    )
                    temp_df['trend_value'] = intercept + slope * temp_df['year']
                    print(f"  Temperature trend: {slope:.4f}°C/year (p={p_value:.4f}, r²={r_value**2:.4f})")

        # Precipitation - Combine all sources with priority order
        if not precip_df.empty:
            print(f"  Creating combined precipitation variable...")

            # Priority order for precipitation:
            # 1. Frost (MET Norway) - prcp_frost
            # 2. SeNorge - prcp_senorge
            # 3. NOAA - prcp_noaa
            # 4. ERA5 - prcp_era5
            # 5. CHELSA - prcp_chelsa
            # 6. E-OBS - prcp_eobs

            # Start with an empty combined column
            precip_df['prcp_combined'] = np.nan

            # Apply each source in priority order
            if 'prcp_frost' in precip_df.columns:
                precip_df['prcp_combined'] = precip_df['prcp_frost']

            # If there are still NaN values, fill with SeNorge data
            if 'prcp_senorge' in precip_df.columns:
                precip_df['prcp_combined'] = precip_df['prcp_combined'].fillna(precip_df['prcp_senorge'])

            # If there are still NaN values, use NOAA data
            if 'prcp_noaa' in precip_df.columns:
                precip_df['prcp_combined'] = precip_df['prcp_combined'].fillna(precip_df['prcp_noaa'])

            # If there are still NaN values, use ERA5 data
            if 'prcp_era5' in precip_df.columns:
                precip_df['prcp_combined'] = precip_df['prcp_combined'].fillna(precip_df['prcp_era5'])

            # If there are still NaN values, use CHELSA data
            if 'prcp_chelsa' in precip_df.columns:
                precip_df['prcp_combined'] = precip_df['prcp_combined'].fillna(precip_df['prcp_chelsa'])

            # If there are still NaN values, use E-OBS data
            if 'prcp_eobs' in precip_df.columns:
                precip_df['prcp_combined'] = precip_df['prcp_combined'].fillna(precip_df['prcp_eobs'])

            # Calculate precipitation trend
            if 'prcp_combined' in precip_df.columns:
                precip_df = precip_df.sort_values('year')
                years = precip_df['year'].values
                precips = precip_df['prcp_combined'].values
                # Remove NaN values for regression
                valid_idx = ~np.isnan(precips)
                if sum(valid_idx) > 1:  # Need at least 2 points for regression
                    slope, intercept, r_value, p_value, std_err = stats.linregress(
                        years[valid_idx], precips[valid_idx]
                    )
                    precip_df['trend_value'] = intercept + slope * precip_df['year']
                    print(f"  Precipitation trend: {slope:.2f}mm/year (p={p_value:.4f}, r²={r_value**2:.4f})")

        # Snow - Combine all sources with priority order
        if not snow_df.empty:
            print(f"  Creating combined snow variable...")

            # Priority order for snow depth:
            # 1. Frost (MET Norway) - snow_depth_frost
            # 2. SeNorge - snow_depth_senorge
            # 3. NOAA - snowdepth_noaa

            # Start with an empty combined column
            snow_df['snow_depth_combined'] = np.nan

            # Apply each source in priority order
            if 'snow_depth_frost' in snow_df.columns:
                snow_df['snow_depth_combined'] = snow_df['snow_depth_frost']

            # If there are still NaN values, fill with SeNorge data
            if 'snow_depth_senorge' in snow_df.columns:
                snow_df['snow_depth_combined'] = snow_df['snow_depth_combined'].fillna(snow_df['snow_depth_senorge'])

            # If there are still NaN values, use NOAA data
            if 'snowdepth_noaa' in snow_df.columns:
                snow_df['snow_depth_combined'] = snow_df['snow_depth_combined'].fillna(snow_df['snowdepth_noaa'])

        # Wind - Combine all sources with priority order
        if not wind_df.empty:
            print(f"  Creating combined wind variable...")

            # Priority order for wind:
            # 1. Frost (MET Norway) - wind_frost
            # 2. NOAA - wind_noaa

            # Start with an empty combined column
            wind_df['wind_combined'] = np.nan

            # Apply each source in priority order
            if 'wind_frost' in wind_df.columns:
                wind_df['wind_combined'] = wind_df['wind_frost']

            # If there are still NaN values, use NOAA data
            if 'wind_noaa' in wind_df.columns:
                wind_df['wind_combined'] = wind_df['wind_combined'].fillna(wind_df['wind_noaa'])

        # Store integrated data
        integrated_results[site_name]['integrated_data']['temperature'] = temp_df
        integrated_results[site_name]['integrated_data']['precipitation'] = precip_df
        integrated_results[site_name]['integrated_data']['wind'] = wind_df
        integrated_results[site_name]['integrated_data']['snow'] = snow_df
        integrated_results[site_name]['integrated_data']['cloudiness'] = cloud_df

    return integrated_results

In [ ]:
def export_integrated_data(integrated_results, output_dir="/content/climate_outputs"):
    """
    Export and visualize the integrated climate data.

    Parameters:
    ----------
    integrated_results : dict
        Dictionary with integrated climate data
    output_dir : str, optional
        Directory to save output files
    """
    os.makedirs(output_dir, exist_ok=True)

    # Create subdirectories for different data types
    integrated_dir = f"{output_dir}/integrated"
    plots_dir = f"{output_dir}/plots"

    os.makedirs(integrated_dir, exist_ok=True)
    os.makedirs(plots_dir, exist_ok=True)

    for site_name, data in integrated_results.items():
        print(f"\n===== Exporting Integrated Data for {site_name} =====")

        # Create site-specific subdirectories
        site_dir = f"{integrated_dir}/{site_name}"
        site_plots_dir = f"{plots_dir}/{site_name}"

        os.makedirs(site_dir, exist_ok=True)
        os.makedirs(site_plots_dir, exist_ok=True)

        # Export metadata
        if 'metadata' in data:
            metadata_file = f"{site_dir}/metadata.json"
            # Convert metadata to a format that can be saved as JSON
            metadata = {
                'site_name': data['metadata']['site_name'],
                'latitude': float(data['metadata']['latitude']),
                'longitude': float(data['metadata']['longitude']),
                'sources_used': data['metadata']['sources_used']
            }

            if data['metadata']['noaa_station']:
                metadata['noaa_station'] = {
                    'id': data['metadata']['noaa_station']['id'],
                    'name': data['metadata']['noaa_station']['name'],
                    'latitude': float(data['metadata']['noaa_station']['latitude']),
                    'longitude': float(data['metadata']['noaa_station']['longitude'])
                }

            # Save metadata
            import json
            with open(metadata_file, 'w') as f:
                json.dump(metadata, f, indent=4)
            print(f"Metadata saved to: {metadata_file}")

        # Temperature data
        temp_df = data['integrated_data']['temperature']
        if not temp_df.empty:
            temp_file = f"{site_dir}/temperature.csv"
            temp_df.to_csv(temp_file, index=False)
            print(f"Temperature data saved to: {temp_file}")

            # Plot temperature data
            if 'temp_combined' in temp_df.columns:
                plt.figure(figsize=(12, 6))

                # Plot the actual temperature data
                plt.plot(temp_df['year'], temp_df['temp_combined'], marker='o', linestyle='-',
                         color='blue', label='Combined Temperature')

                # If we have tmax and tmin from NOAA, plot those too
                if 'tmax_noaa' in temp_df.columns:
                    plt.plot(temp_df['year'], temp_df['tmax_noaa'], marker='^', linestyle='--',
                             color='red', alpha=0.7, label='NOAA Max Temp')
                if 'tmin_noaa' in temp_df.columns:
                    plt.plot(temp_df['year'], temp_df['tmin_noaa'], marker='v', linestyle='--',
                             color='blue', alpha=0.7, label='NOAA Min Temp')

                # If we have ERA5 temperature, plot that too
                if 'temp_era5' in temp_df.columns:
                    plt.plot(temp_df['year'], temp_df['temp_era5'], marker='s', linestyle=':',
                             color='green', alpha=0.7, label='ERA5 Temperature')

                # If we calculated a trend, plot it
                if 'trend_value' in temp_df.columns:
                    plt.plot(temp_df['year'], temp_df['trend_value'], linestyle='-',
                             color='black', alpha=0.7, label='Trend')

                plt.title(f'Annual Temperature for {site_name}')
                plt.xlabel('Year')
                plt.ylabel('Temperature (°C)')
                plt.legend()
                plt.grid(True, alpha=0.3)

                # Add a horizontal line at 0°C
                plt.axhline(y=0, color='gray', linestyle='-', alpha=0.5)

                # Save the plot
                temp_plot_file = f"{site_plots_dir}/temperature_trend.png"
                plt.savefig(temp_plot_file, dpi=300)
                plt.close()
                print(f"Temperature plot saved to: {temp_plot_file}")

        # Precipitation data
        precip_df = data['integrated_data']['precipitation']
        if not precip_df.empty:
            precip_file = f"{site_dir}/precipitation.csv"
            precip_df.to_csv(precip_file, index=False)
            print(f"Precipitation data saved to: {precip_file}")

            # Plot precipitation data
            if 'prcp_combined' in precip_df.columns:
                plt.figure(figsize=(12, 6))

                # Plot the actual precipitation data
                plt.plot(precip_df['year'], precip_df['prcp_combined'], marker='o', linestyle='-',
                         color='blue', label='Combined Precipitation')

                # If we have NOAA precipitation, plot that too
                if 'prcp_noaa' in precip_df.columns:
                    plt.plot(precip_df['year'], precip_df['prcp_noaa'], marker='^', linestyle='--',
                             color='red', alpha=0.7, label='NOAA Precipitation')

                # If we have ERA5 precipitation, plot that too
                if 'prcp_era5' in precip_df.columns:
                    plt.plot(precip_df['year'], precip_df['prcp_era5'], marker='s', linestyle=':',
                             color='green', alpha=0.7, label='ERA5 Precipitation')

                # If we calculated a trend, plot it
                if 'trend_value' in precip_df.columns:
                    plt.plot(precip_df['year'], precip_df['trend_value'], linestyle='-',
                             color='black', alpha=0.7, label='Trend')

                plt.title(f'Annual Precipitation for {site_name}')
                plt.xlabel('Year')
                plt.ylabel('Precipitation (mm)')
                plt.legend()
                plt.grid(True, alpha=0.3)

                # Save the plot
                precip_plot_file = f"{site_plots_dir}/precipitation_trend.png"
                plt.savefig(precip_plot_file, dpi=300)
                plt.close()
                print(f"Precipitation plot saved to: {precip_plot_file}")

        # Wind data
        wind_df = data['integrated_data']['wind']
        if not wind_df.empty:
            wind_file = f"{site_dir}/wind.csv"
            wind_df.to_csv(wind_file, index=False)
            print(f"Wind data saved to: {wind_file}")

            # Plot wind data
            if 'wind_noaa' in wind_df.columns:
                plt.figure(figsize=(12, 6))
                plt.plot(wind_df['year'], wind_df['wind_noaa'], marker='o', linestyle='-', color='blue')
                plt.title(f'Annual Average Wind Speed for {site_name}')
                plt.xlabel('Year')
                plt.ylabel('Wind Speed (m/s)')
                plt.grid(True, alpha=0.3)

                # Save the plot
                wind_plot_file = f"{site_plots_dir}/wind_trend.png"
                plt.savefig(wind_plot_file, dpi=300)
                plt.close()
                print(f"Wind plot saved to: {wind_plot_file}")

        # Snow data
        snow_df = data['integrated_data']['snow']
        if not snow_df.empty:
            snow_file = f"{site_dir}/snow.csv"
            snow_df.to_csv(snow_file, index=False)
            print(f"Snow data saved to: {snow_file}")

            # Plot snow data if we have snowfall or snow depth
            if 'snowfall_noaa' in snow_df.columns or 'snowdepth_noaa' in snow_df.columns:
                plt.figure(figsize=(12, 6))

                if 'snowfall_noaa' in snow_df.columns:
                    plt.plot(snow_df['year'], snow_df['snowfall_noaa'], marker='o', linestyle='-',
                             color='blue', label='Snowfall')

                if 'snowdepth_noaa' in snow_df.columns:
                    plt.plot(snow_df['year'], snow_df['snowdepth_noaa'], marker='^', linestyle='--',
                             color='red', label='Snow Depth')

                plt.title(f'Annual Snow Data for {site_name}')
                plt.xlabel('Year')
                plt.ylabel('Snow (mm)')
                plt.legend()
                plt.grid(True, alpha=0.3)

                # Save the plot
                snow_plot_file = f"{site_plots_dir}/snow_trend.png"
                plt.savefig(snow_plot_file, dpi=300)
                plt.close()
                print(f"Snow plot saved to: {snow_plot_file}")

        # Cloudiness data
        cloud_df = data['integrated_data']['cloudiness']
        if not cloud_df.empty:
            cloud_file = f"{site_dir}/cloudiness.csv"
            cloud_df.to_csv(cloud_file, index=False)
            print(f"Cloudiness data saved to: {cloud_file}")

            # Plot cloudiness data
            if 'cloud_noaa' in cloud_df.columns:
                plt.figure(figsize=(12, 6))
                plt.plot(cloud_df['year'], cloud_df['cloud_noaa'], marker='o', linestyle='-', color='blue')
                plt.title(f'Annual Average Cloudiness for {site_name}')
                plt.xlabel('Year')
                plt.ylabel('Cloudiness (%)')
                plt.grid(True, alpha=0.3)

                # Save the plot
                cloud_plot_file = f"{site_plots_dir}/cloudiness_trend.png"
                plt.savefig(cloud_plot_file, dpi=300)
                plt.close()
                print(f"Cloudiness plot saved to: {cloud_plot_file}")

        # Create a comprehensive integrated dataset
        print(f"Creating comprehensive dataset for {site_name}...")

        # Start with temperature data as the base
        if not temp_df.empty:
            comprehensive_df = temp_df.copy()

            # Add precipitation data
            if not precip_df.empty:
                # Only keep the year and combined precipitation columns
                precip_cols = ['year', 'prcp_combined'] if 'prcp_combined' in precip_df.columns else ['year']
                precip_cols.extend([col for col in precip_df.columns if col.startswith('prcp_') and col != 'prcp_combined'])
                precip_subset = precip_df[precip_cols].copy()

                # Merge with existing data
                comprehensive_df = pd.merge(comprehensive_df, precip_subset, on='year', how='outer')

            # Add wind data
            if not wind_df.empty:
                # Only keep the year and wind columns
                wind_cols = ['year'] + [col for col in wind_df.columns if col.startswith('wind_')]
                wind_subset = wind_df[wind_cols].copy()

                # Merge with existing data
                comprehensive_df = pd.merge(comprehensive_df, wind_subset, on='year', how='outer')

            # Add snow data
            if not snow_df.empty:
                # Only keep the year and snow columns
                snow_cols = ['year'] + [col for col in snow_df.columns if col.startswith('snow')]
                snow_subset = snow_df[snow_cols].copy()

                # Merge with existing data
                comprehensive_df = pd.merge(comprehensive_df, snow_subset, on='year', how='outer')

            # Add cloudiness data
            if not cloud_df.empty:
                # Only keep the year and cloudiness columns
                cloud_cols = ['year'] + [col for col in cloud_df.columns if col.startswith('cloud')]
                cloud_subset = cloud_df[cloud_cols].copy()

                # Merge with existing data
                comprehensive_df = pd.merge(comprehensive_df, cloud_subset, on='year', how='outer')

            # Sort by year
            comprehensive_df = comprehensive_df.sort_values('year')

            # Save the comprehensive dataset
            comprehensive_file = f"{site_dir}/comprehensive_climate_data.csv"
            comprehensive_df.to_csv(comprehensive_file, index=False)
            print(f"Comprehensive climate data saved to: {comprehensive_file}")
        else:
            print(f"Warning: No temperature data available for {site_name}, skipping comprehensive dataset")

In [ ]:
def create_zip_download(output_dir, zip_filename="climate_data_outputs.zip"):
    print(f"Creating zip file of all outputs for download: {zip_filename}")
    with zipfile.ZipFile(zip_filename, 'w') as zipf:
        for root, _, files in os.walk(output_dir):
            for file in files:
                file_path = os.path.join(root, file)
                # Add file to zip with a relative path
                zipf.write(file_path, os.path.relpath(file_path, os.path.dirname(output_dir)))

    print(f"Zip file created: {zip_filename}")
    return zip_filename

In [ ]:
def run_integrated_climate_analysis(sites_df, start_year=1979, end_year=None, output_dir="/content/climate_outputs"):
    if end_year is None:
        end_year = datetime.now().year

    print("\n===== INTEGRATED CLIMATE DATA ANALYSIS FOR NORWEGIAN GLACIERS =====")
    print(f"Start year: {start_year}")
    print(f"End year: {end_year}")
    print(f"Output directory: {output_dir}")

    # 1. Check API credentials
    print("\nStep 1: Setting up API credentials")

    # MET Norway Frost API (highest priority for Norwegian glaciers)
    frost_client_id = input("Enter your MET Norway Frost API client ID (leave blank to skip Frost data): ")

    # NOAA API
    noaa_token = input("Enter your NOAA API token (leave blank to skip NOAA data): ")

    # CDS API
    use_cds = input("Use Climate Data Store (CDS) ERA5 data? (y/n): ").lower() == 'y'
    cds_setup = False
    if use_cds:
        cds_setup = setup_cds_api()

    print("\n==== DEBUGGING: API CREDENTIALS ====")
    print(f"Frost client ID provided: {'Yes' if frost_client_id else 'No'}")
    print(f"NOAA token provided: {'Yes' if noaa_token else 'No'}")
    print(f"CDS setup successful: {'Yes' if cds_setup else 'No'}")

    # 2. Verify data availability
    print("\nStep 2: Verifying data availability for each site and source")
    availability = verify_data_availability(sites_df, noaa_token, cds_setup, frost_client_id)

    # Display availability summary
    print("\nData Availability Summary:")
    for site, sources in availability.items():
        print(f"\n{site}:")

        # Frost (highest priority for Norwegian sites)
        if sources['frost']['available']:
            print("  MET Norway Frost: Available")
            print("    Variables:")
            for var, avail in sources['frost']['variables'].items():
                print(f"      {var}: {'✓' if avail else '✗'}")
        else:
            print("  MET Norway Frost: Not available")

        # SeNorge (high priority for Norwegian sites)
        if sources['senorge']['available']:
            print("  SeNorge: Available")
            print("    Variables:")
            for var, avail in sources['senorge']['variables'].items():
                print(f"      {var}: {'✓' if avail else '✗'}")
        else:
            print("  SeNorge: Not available")

        # NVE (specific to glaciers)
        if sources['nve']['available']:
            print("  NVE: Available")
            print("    Variables:")
            for var, avail in sources['nve']['variables'].items():
                print(f"      {var}: {'✓' if avail else '✗'}")
        else:
            print("  NVE: Not available")

        # NOAA
        if sources['noaa']['available']:
            print("  NOAA: Available")
            print("    Variables:")
            for var, avail in sources['noaa']['variables'].items():
                print(f"      {var}: {'✓' if avail else '✗'}")
        else:
            print("  NOAA: Not available")

        # ERA5
        if sources['era5']['available']:
            print("  ERA5: Available")
            print("    Variables:")
            for var, avail in sources['era5']['variables'].items():
                print(f"      {var}: {'✓' if avail else '✗'}")
        else:
            print("  ERA5: Not available")

        # CHELSA
        if sources['chelsa']['available']:
            print("  CHELSA: Available")
            print("    Variables:")
            for var, avail in sources['chelsa']['variables'].items():
                print(f"      {var}: {'✓' if avail else '✗'}")
        else:
            print("  CHELSA: Not available")

        # E-OBS
        if sources['eobs']['available']:
            print("  E-OBS: Available")
            print("    Variables:")
            for var, avail in sources['eobs']['variables'].items():
                print(f"      {var}: {'✓' if avail else '✗'}")
        else:
            print("  E-OBS: Not available")

    # 3. Fetch data from multiple sources
    print("\nStep 3: Fetching data from all available sources")
    integrated_results = get_integrated_climate_data(sites_df, availability,
                                                  start_year, end_year,
                                                  noaa_token, frost_client_id,
                                                  cds_setup, output_dir)

    # 4. Integrate data from different sources
    print("\nStep 4: Integrating data from different sources")
    integrated_results = integrate_climate_data(integrated_results)

    # 5. Export and visualize the integrated data
    print("\nStep 5: Exporting and visualizing the integrated data")
    export_integrated_data(integrated_results, output_dir)

    print("\n===== ANALYSIS COMPLETE =====")
    print(f"All results saved to: {output_dir}")

    # Create a zip file with all outputs
    print("\nCreating zip file for download...")
    zip_file = create_zip_download(output_dir)

    return integrated_results

In [ ]:
def main():
    """Main function to execute the script."""
    print("==== INTEGRATED CLIMATE DATA FOR NORWEGIAN GLACIER SITES ====\n")

    # Define local output directory
    output_dir = "/content/climate_outputs"
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    # Try to load glacier sites
    print("Step 1: Load glacier study sites")

    # Option: Set default paths for common Norwegian glacier datasets
    default_paths = {
        'jostedal': "/content/drive/MyDrive/Glaciers/jostedal_glaciers.csv",
        'jotunheimen': "/content/drive/MyDrive/Glaciers/jotunheimen_glaciers.csv",
        'folgefonna': "/content/drive/MyDrive/Glaciers/folgefonna_glaciers.csv",
        'svartisen': "/content/drive/MyDrive/Glaciers/svartisen_glaciers.csv",
        'okstindan': "/content/drive/MyDrive/Glaciers/okstindan_glaciers.csv",
        'custom': "/content/drive/MyDrive/Diss/Climate_data_v6/glacier_locations.csv"
    }

    print("Choose a glacier region or load your own dataset:")
    print("1. Jostedalsbreen")
    print("2. Jotunheimen")
    print("3. Folgefonna")
    print("4. Svartisen")
    print("5. Okstindan")
    print("6. Custom (your own dataset)")

    choice = input("Enter your choice (1-6): ")

    if choice == '1':
        default_path = default_paths['jostedal']
        region_name = "Jostedalsbreen"
    elif choice == '2':
        default_path = default_paths['jotunheimen']
        region_name = "Jotunheimen"
    elif choice == '3':
        default_path = default_paths['folgefonna']
        region_name = "Folgefonna"
    elif choice == '4':
        default_path = default_paths['svartisen']
        region_name = "Svartisen"
    elif choice == '5':
        default_path = default_paths['okstindan']
        region_name = "Okstindan"
    else:
        default_path = default_paths['custom']
        region_name = "Custom"

    # Try to load from the selected path
    try:
        sites_df = upload_glacier_sites(default_path)
        print(f"Successfully loaded {region_name} glacier sites from:", default_path)
    except:
        print(f"File not found at {region_name} location. You can either:")
        print("1. Upload a file manually")
        print("2. Enter a file path")

        choice = input("Choose option (1 or 2): ")

        if choice == '1':
            print("Please upload your CSV file with glacier locations:")
            try:
                from google.colab import files
                uploaded = files.upload()
                file_path = list(uploaded.keys())[0]
                sites_df = upload_glacier_sites(file_path)
            except:
                print("Could not upload file. Please try again or use option 2.")
                sys.exit(1)
        else:
            file_path = input("Enter the full path to your CSV file: ")
            try:
                sites_df = upload_glacier_sites(file_path)
            except:
                print("Could not load file. Please check the path and try again.")
                sys.exit(1)

    if sites_df is None:
        print("Failed to load glacier sites. Exiting.")
        sys.exit(1)

    print("\nYour glacier sites:")
    print(sites_df)

    # Get start and end years
    while True:
        try:
            start_year = input("\nEnter start year (default is 1970): ")
            start_year = int(start_year) if start_year else 1970

            end_year = input("Enter end year (default is current year): ")
            end_year = int(end_year) if end_year else datetime.now().year

            if start_year > end_year:
                print("Error: Start year must be before end year.")
                continue

            break
        except ValueError:
            print("Error: Please enter valid years (integers).")

    # Run the integrated analysis
    integrated_results = run_integrated_climate_analysis(sites_df, start_year, end_year, output_dir)

    print("\nAnalysis complete!")
    try:
        from google.colab import files
        print("\nTo download all results as a zip file, run:")
        print("from google.colab import files")
        print("files.download('climate_data_outputs.zip')")
    except:
        print(f"\nAll results are saved in the directory: {output_dir}")
        print(f"A zip file with all results is available at: climate_data_outputs.zip")

if __name__ == "__main__":
    main()

==== INTEGRATED CLIMATE DATA FOR NORWEGIAN GLACIER SITES ====

Step 1: Load glacier study sites
Choose a glacier region or load your own dataset:
1. Jostedalsbreen
2. Jotunheimen
3. Folgefonna
4. Svartisen
5. Okstindan
6. Custom (your own dataset)
Enter your choice (1-6): 6
Successfully loaded Custom glacier sites from: /content/drive/MyDrive/Diss/Climate_data_v6/glacier_locations.csv

Your glacier sites:
   site_name   latitude  longitude  Unnamed: 3  Unnamed: 4  Unnamed: 5
0        g01  68.579515  18.107387         NaN         NaN         NaN
1        g11  66.501116  14.655010         NaN         NaN         NaN
2        g12  65.445231  13.139741         NaN         NaN         NaN
3        g13  65.382565  13.125551         NaN         NaN         NaN
4        g19  61.874247   7.230627         NaN         NaN         NaN
5        g20  67.416720  16.026525         NaN         NaN         NaN
6        g23  69.275026  19.765340         NaN         NaN         NaN
7        g25  69.148559

In [ ]:
from google.colab import files
files.download('climate_data_outputs.zip')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>